# Config

## TIPS Treasury Arbitrage:

```python
def task_config():
    """Create empty directories for data and output if they don't exist"""
    return {
        "actions": ["ipython ./src/settings.py"],
        "targets": [DATA_DIR, OUTPUT_DIR],
        "file_dep": ["./src/settings.py"],
        "clean": [],
    }
```

## Equity Spot Futures:
```python
# Define the paths based on configuration
BASE_DIR = config("BASE_DIR")
DATA_DIR = Path(config("DATA_DIR"))
OUTPUT_DIR = Path(config("OUTPUT_DIR"))
TEMP_DIR = Path(config("TEMP_DIR"))
INPUT_DIR = Path(config("INPUT_DIR"))
# PUBLISH_DIR = Path(config("PUBLISH_DIR"))
PROCESSED_DIR = Path(config("PROCESSED_DIR"))

# Define log file paths
LOG_FILES = [
    TEMP_DIR / "futures_processing.log",
    TEMP_DIR / "ois_processing.log",
    TEMP_DIR / "bloomberg_data_extraction.log"
]

def task_config():
    """Create empty directories for data and output if they don't exist, and ensure log files are created"""
    return {
        "actions": ["ipython ./src/settings.py"],  # This action should ensure directories and files are prepared
        "targets": [
            DATA_DIR, OUTPUT_DIR, TEMP_DIR, INPUT_DIR,  PROCESSED_DIR
        ] + LOG_FILES,  # Include log files in the targets to manage their existence
        "file_dep": ["./src/settings.py"],
        "clean": True,  # This will clean up all directories and log files when 'doit clean' is executed
    }
```


## CIP
```python

DATA_DIR = (config("DATA_DIR"))
OUTPUT_DIR = (config("OUTPUT_DIR"))
MANUAL_DATA_DIR = (config("MANUAL_DATA_DIR"))
PUBLISH_DIR = (config("PUBLISH_DIR"))

def update_bloomberg():
    """Prompt the user and update the BLOOMBERG variable in settings.py."""
    user_input = input("Do you want to run from Bloomberg terminal? (Y/N): ").strip().upper()
    new_value = "True" if user_input == "Y" else "False"

    settings_path = Path("src/settings.py")

    # Read settings.py
    with open(settings_path, "r") as f:
        lines = f.readlines()

    # Modify the BLOOMBERG variable if it exists, otherwise append it
    found = False
    for i, line in enumerate(lines):
        if line.strip().startswith("BLOOMBERG ="):
            lines[i] = f"BLOOMBERG = {new_value}\n"
            found = True
            break

    if not found:
        lines.append(f"\nBLOOMBERG = {new_value}\n")

    # Write updated settings back to file
    with open(settings_path, "w") as f:
        f.writelines(lines)

    import importlib
    importlib.reload(src.settings)

    if new_value == "True":
        try:
            from xbbg import blp  # Attempt to import xbbg
        except ModuleNotFoundError:
            print("No Bloomberg terminal found on device.")

def task_BLOOMBERG():
    """PyDoit task to update Bloomberg mode in settings.py."""
    return {
        "actions": [update_bloomberg],
        "verbosity": 2,  # Ensure output is shown
    }

def task_config():
    """Create directories for data and output if they don't exist."""
    def create_dirs():
        os.makedirs(DATA_DIR, exist_ok=True)
        os.makedirs(MANUAL_DATA_DIR, exist_ok=True)
        os.makedirs(OUTPUT_DIR, exist_ok=True)
        os.makedirs(PUBLISH_DIR, exist_ok=True)

    return {
        "actions": [create_dirs],
        "targets": [str(DATA_DIR), str(OUTPUT_DIR), str(MANUAL_DATA_DIR), str(PUBLISH_DIR)],
        "clean": True,
    }
```

## Market Expectations
```python
BASE_DIR = config("BASE_DIR")
DATA_DIR = config("DATA_DIR")
MANUAL_DATA_DIR = config("MANUAL_DATA_DIR")
OUTPUT_DIR = config("OUTPUT_DIR")
#OS_TYPE = config("OS_TYPE")
PUBLISH_DIR = config("PUBLISH_DIR")
USER = config("USER")
PLOTS_DIR = config("PLOTS_DIR")
TABLES_DIR = config("TABLES_DIR")


def task_config():
    """Create empty directories for data, output, plots, and tables if they don't exist"""
    return {
        "actions": ["ipython ./src/settings.py"],
        "targets": [DATA_DIR, OUTPUT_DIR, PLOTS_DIR, TABLES_DIR],
        "file_dep": ["./src/settings.py"],
        "clean": [],  # Don't clean these files by default.
    }
```

In [ ]:
from pathlib import Path
from platform import system
from decouple import config as _config
from pandas import to_datetime
BLOOMBERG = False

def get_os():
    os_name = system()
    if os_name == "Windows":
        return "windows"
    elif os_name in ("Darwin", "Linux"):
        return "nix"
    else:
        return "unknown"

def if_relative_make_abs(path):
    """If a relative path is given, make it absolute, assuming
    that it is relative to the project root directory (BASE_DIR)

    Example
    -------
    ```
    >>> if_relative_make_abs(Path('_data'))
    WindowsPath('C:/Users/jdoe/GitRepositories/blank_project/_data')

    >>> if_relative_make_abs(Path("C:/Users/jdoe/GitRepositories/blank_project/_output"))
    WindowsPath('C:/Users/jdoe/GitRepositories/blank_project/_output')
    ```
    """
    path = Path(path)
    if path.is_absolute():
        abs_path = path.resolve()
    else:
        abs_path = (d["BASE_DIR"] / path).resolve()
    return abs_path

d = {}
d["OS_TYPE"] = get_os()

# Absolute path to root directory of the project
d["BASE_DIR"] = Path(__file__).absolute().parent.parent

# fmt: off
## Other .env variables
d["START_DATE"] = _config("START_DATE", default="1913-01-01", cast=to_datetime)
d["END_DATE"] = _config("END_DATE", default="2024-01-01", cast=to_datetime)
d["PIPELINE_DEV_MODE"] = _config("PIPELINE_DEV_MODE", default=True, cast=bool)
d["PIPELINE_THEME"] = _config("PIPELINE_THEME", default="pipeline")

d["USING_XBBG"]        = _config("USING_XBBG", default=False, cast=bool)

## Paths
d["DATA_DIR"] = if_relative_make_abs(_config('DATA_DIR', default=Path('_data'), cast=Path))
d["MANUAL_DATA_DIR"] = if_relative_make_abs(_config('MANUAL_DATA_DIR', default=Path('data_manual'), cast=Path))
d["OUTPUT_DIR"] = if_relative_make_abs(_config('OUTPUT_DIR', default=Path('_output'), cast=Path))
d["PUBLISH_DIR"] = if_relative_make_abs(_config('PUBLISH_DIR', default=Path('_output/publish'), cast=Path))
# fmt: on

#Equity spot
d["INPUT_DIR"]     = if_relative_make_abs(_config('INPUT_DIR', default=Path('_data/input'), cast=Path))
d["PROCESSED_DIR"] = if_relative_make_abs(_config('PROCESSED_DIR', default=Path('_data/processed'), cast=Path))
d["TEMP_DIR"]      = if_relative_make_abs(_config('TEMP_DIR', default=Path('_output/temp'), cast=Path))

#CIP
d["REPORTS_DIR"] = if_relative_make_abs(_config("REPORTS_DIR", default=Path("reports"), cast=Path))

# Market Expectations
d["PLOTS_DIR"] = if_relative_make_abs(_config('PLOTS_DIR', default=Path('reports/plots'), cast=Path))
d["TABLES_DIR"] = if_relative_make_abs(_config('TABLES_DIR', default=Path('reports/tables'), cast=Path))


## Name of Stata Executable in path
if d["OS_TYPE"] == "windows":
    d["STATA_EXE"] = _config("STATA_EXE", default="StataMP-64.exe")
elif d["OS_TYPE"] == "nix":
    d["STATA_EXE"] = _config("STATA_EXE", default="stata-mp")
else:
    raise ValueError("Unknown OS type")

def create_dirs():
    ## If they don't exist, create the _data and _output directories
    d["DATA_DIR"].mkdir(parents=True, exist_ok=True)
    d["OUTPUT_DIR"].mkdir(parents=True, exist_ok=True)
    # (d["BASE_DIR"] / "_docs").mkdir(parents=True, exist_ok=True)

    #Equity spot
    d["TEMP_DIR"].mkdir(parents=True, exist_ok=True)
    d["INPUT_DIR"].mkdir(parents=True, exist_ok=True)
    # d["PUBLISH_DIR"].mkdir(parents=True, exist_ok=True)
    d["PROCESSED_DIR"].mkdir(parents=True, exist_ok=True)

    # If you'd like to ensure these log files exist (touch them):
    for log_filename in (
        "futures_processing.log",
        "ois_processing.log",
        "bloomberg_data_extraction.log",
    ):
        log_file_path = d["TEMP_DIR"] / log_filename
        log_file_path.touch(exist_ok=True)

    # CIP
    d["REPORTS_DIR"].mkdir(parents=True, exist_ok=True)

def config(*args, **kwargs):
    """
    Retrieve configuration variables. 
    Checks `d` first. If not found, falls back to .env via decouple.config.
    """
    key = args[0]
    default = kwargs.get("default", None)
    cast = kwargs.get("cast", None)
    if key in d:
        var = d[key]
        # If a default was passed but we already have a value in d, raise an error
        if default is not None:
            raise ValueError(f"Default for {key} already exists. Check your settings.py file.")
        if cast is not None:
            # If cast is requested, check that it wouldn't change the type
            if not isinstance(var, cast):
                # or if we want to actually recast:
                try:
                    new_var = cast(var)
                except Exception as e:
                    raise ValueError(f"Could not cast {key} to {cast}: {e}") from e
                if type(new_var) is not type(var):
                    raise ValueError(f"Type for {key} differs. Check your settings.py file.")
        return var
    else:
        return _config(*args, **kwargs)

if __name__ == "__main__":
    create_dirs()

# Data Loading


## TIPS Treasury Arbitrage:
Manual data: 

```python
def task_pull_fed_yield_curve():
    """ """
    file_dep = [
        "./src/pull_fed_yield_curve.py",
    ]
    targets = [
        DATA_DIR / "fed_yield_curve_all.parquet",
        DATA_DIR / "fed_yield_curve.parquet",
    ]

    return {
        "actions": [
            "ipython ./src/pull_fed_yield_curve.py",
        ],
        "targets": targets,
        "file_dep": file_dep,
        "clean": [],
    }
```

In [ ]:
import pandas as pd
import requests
from io import BytesIO
from pathlib import Path

from settings import config
DATA_DIR = config('DATA_DIR')


def pull_fed_yield_curve():
    """
    Download the latest yield curve from the Federal Reserve
    
    This is the published data using Gurkaynak, Sack, and Wright (2007) model
    
    load in as: 
    "Treasury_SF_10Y",
    "Treasury_SF_02Y",
    "Treasury_SF_20Y",
    "Treasury_SF_03Y",
    "Treasury_SF_30Y",
    "Treasury_SF_05Y",
    """
    
    url = "https://www.federalreserve.gov/data/yield-curve-tables/feds200628.csv"
    response = requests.get(url)
    pdf_stream = BytesIO(response.content)
    df_all = pd.read_csv(pdf_stream, skiprows=9, index_col=0, parse_dates=True)

    cols = ['SVENY' + str(i).zfill(2) for i in range(1, 31)]
    df = df_all[cols]
    return df_all, df

def load_fed_yield_curve_all(data_dir=DATA_DIR):
    path = data_dir / "fed_yield_curve_all.parquet"
    _df = pd.read_parquet(path)
    
    # Select the specific columns. Note: SVENY03 is included so that
    # we can rename to "Treasury_SF_03Y" as requested.
    selected_cols = ['SVENY02', 'SVENY03', 'SVENY05', 'SVENY10', 'SVENY20', 'SVENY30']
    _df = _df[selected_cols]
    
    # Rename the columns to the desired names.
    rename_mapping = {
        'SVENY10': 'Treasury_SF_10Y',
        'SVENY02': 'Treasury_SF_02Y',
        'SVENY20': 'Treasury_SF_20Y',
        'SVENY03': 'Treasury_SF_03Y',
        'SVENY30': 'Treasury_SF_30Y',
        'SVENY05': 'Treasury_SF_05Y'
    }
    _df = _df.rename(columns=rename_mapping)
    
    return _df

def load_fed_yield_curve(data_dir=DATA_DIR):
    path = data_dir / "fed_yield_curve.parquet"
    _df = pd.read_parquet(path)
    return _df

def _demo():
    _df = pull_fed_yield_curve(data_dir=DATA_DIR)
    

if __name__ == "__main__":
    df_all, df = pull_fed_yield_curve()
    path = Path(DATA_DIR) / "fed_yield_curve_all.parquet"
    df_all.to_parquet(path)
    path = Path(DATA_DIR) / "fed_yield_curve.parquet"
    df.to_parquet(path)

In [6]:
import pandas as pd
from settings import config

LOCAL_SERIAL_DATA_DIR    = config("LOCAL_SERIAL_DATA_DIR")

def local_parquet_head(filepath, rows=10):
    """
    Reads a local Parquet file and returns the first `rows` rows as a DataFrame.
    """
    try:
        df = pd.read_parquet(filepath, engine='pyarrow')  # or 'fastparquet'
        return df.head(rows)
    except Exception as e:
        print(f"Error reading parquet file: {e}")
        return None

# Example usage
file_path = 'fed_yield_curve.parquet'  # Replace with your local path
df_head = local_parquet_head(LOCAL_SERIAL_DATA_DIR+'/'+file_path)

if df_head is not None:
    print(df_head)


            SVENY01  SVENY02  SVENY03  SVENY04  SVENY05  SVENY06  SVENY07  \
Date                                                                        
1961-06-14   2.9825   3.3771   3.5530   3.6439   3.6987   3.7351   3.7612   
1961-06-15   2.9941   3.4137   3.5981   3.6930   3.7501   3.7882   3.8154   
1961-06-16   3.0012   3.4142   3.5994   3.6953   3.7531   3.7917   3.8192   
1961-06-19   2.9949   3.4386   3.6252   3.7199   3.7768   3.8147   3.8418   
1961-06-20   2.9833   3.4101   3.5986   3.6952   3.7533   3.7921   3.8198   
1961-06-21   2.9993   3.4236   3.6132   3.7107   3.7694   3.8085   3.8364   
1961-06-22   2.9837   3.4036   3.5976   3.6981   3.7587   3.7990   3.8279   
1961-06-23   2.9749   3.3706   3.5725   3.6816   3.7478   3.7921   3.8237   
1961-06-26   2.9563   3.3623   3.5678   3.6784   3.7455   3.7903   3.8224   
1961-06-27   2.9666   3.3593   3.5641   3.6778   3.7477   3.7945   3.8280   

            SVENY08  SVENY09  SVENY10  ...  SVENY21  SVENY22  SVENY23  \
Da

```python
def task_pull_fed_tips_yield_curve():
    """ """
    file_dep = [
        "./src/pull_fed_tips_yield_curve.py",
    ]
    targets = [
        DATA_DIR / "fed_tips_yield_curve.parquet",
    ]

    return {
        "actions": [
            "ipython ./src/pull_fed_tips_yield_curve.py",
        ],
        "targets": targets,
        "file_dep": file_dep,
        "clean": [],
    }
```

In [ ]:
from settings import config
DATA_DIR = config('DATA_DIR')

# Define the URL for the TIPS yield data
TIPS_URL = "https://www.federalreserve.gov/data/yield-curve-tables/feds200805.csv"

def pull_fed_tips_yield_curve():
    """
    Download and process the latest zero-coupon TIPS yield curve from the Federal Reserve.

    Expected CSV structure:
    - Metadata in the first 19 rows (to be skipped).
    - 'Date' column in YMD format.
    - TIPS yield columns named 'TIPSY02', 'TIPSY05', 'TIPSY10', 'TIPSY20'.
    - Yield values are in percentage terms and must be converted to decimals.

    Returns:
        pd.DataFrame: Processed TIPS yield data.
    """
    # Fetch the data from the Federal Reserve
    response = requests.get(TIPS_URL)
    if response.status_code != 200:
        raise Exception(f"Failed to fetch TIPS data: HTTP {response.status_code}")
    
    # Read CSV while skipping the first 19 rows (metadata)
    df = pd.read_csv(BytesIO(response.content), skiprows=18)
    
    return df

def save_tips_yield_curve(df, data_dir):
    """
    Save the TIPS yield curve DataFrame to a parquet file.
    """
    path = Path(data_dir) / "fed_tips_yield_curve.parquet"
    df.to_parquet(path)

def load_tips_yield_curve(data_dir):
    """
    Load the TIPS yield curve DataFrame from a parquet file.
    Selects and renames the following columns:
    
    Source columns: TIPSY02, TIPSY05, TIPSY10, TIPSY20, TIPSY30
    Target columns: ['TIPS_Treasury_02Y', 'TIPS_Treasury_05Y', 'TIPS_Treasury_10Y', 'TIPS_Treasury_20Y']
    
    Note: TIPSY30 is ignored since only four target columns are provided.
    """
    path = Path(data_dir) / "fed_tips_yield_curve.parquet"
    df = pd.read_parquet(path)
    
    # Select only the required columns (ignoring TIPSY30)
    selected_cols = ['TIPSY02', 'TIPSY05', 'TIPSY10', 'TIPSY20']
    df = df[selected_cols]
    
    # Rename the selected columns as specified.
    rename_mapping = {
        'TIPSY02': 'TIPS_Treasury_02Y',
        'TIPSY05': 'TIPS_Treasury_05Y',
        'TIPSY10': 'TIPS_Treasury_10Y',
        'TIPSY20': 'TIPS_Treasury_20Y'
    }
    df = df.rename(columns=rename_mapping)
    
    return df

# Example usage
if __name__ == "__main__":
    tips_df = pull_fed_tips_yield_curve()
    save_tips_yield_curve(tips_df, DATA_DIR)

```python
def task_pull_bloomberg_treasury_inflation_swaps():
    """Run pull_bloomberg_treasury_inflation_swaps only if treasury_inflation_swaps.csv is not present in OUTPUT_DIR."""
    from pathlib import Path  # ensure Path is available
    output_dir = Path(OUTPUT_DIR)

    if not output_dir.exists():
        output_dir.mkdir(parents=True, exist_ok=True)
        
    if not any(output_dir.iterdir()):
        # Only yield the nested task if the CSV file is not present.
        yield {
            "name": "run",
            "actions": ["ipython ./src/pull_bloomberg_treasury_inflation_swaps.py"],
            "file_dep": ["./src/pull_bloomberg_treasury_inflation_swaps.py"],
            "targets": [DATA_DIR / "treasury_inflation_swaps.parquet"],
            "clean": [],
        }
    else:
        print("treasury_inflation_swaps.csv exists in OUTPUT_DIR; skipping task_pull_bloomberg_treasury_inflation_swaps")
```

In [ ]:
from xbbg import blp
from decouple import config

OUTPUT_DIR = config("OUTPUT_DIR")
START_DATE = config("START_DATE", "2020-01-01")
END_DATE = config("END_DATE", "2025-01-01")

def pull_treasury_inflation_swaps(
    start_date=START_DATE,
    end_date=END_DATE,
    output_path="treasury_inflation_swaps.csv"
):
    """
    Connects to Bloomberg via xbbg, pulls historical daily prices for USD
    Treasury Inflation Swaps, and saves them to a CSV with columns matching
    the provided treasury_inflation_swaps.csv file.

    :param start_date: Start date in 'YYYY-MM-DD' format (str).
    :param end_date: End date in 'YYYY-MM-DD' format (str).
    :param output_path: Path to save the resulting CSV file.
    :return: A pandas DataFrame containing the replicated data.
    """

    # Tickers to replicate. Adjust as needed for 1M, 3M, 6M, etc.
    tickers = [
        "USSWIT1 BGN Curncy",   # 1Y
        "USSWIT2 BGN Curncy",   # 2Y
        "USSWIT3 BGN Curncy",   # 3Y
        "USSWIT4 BGN Curncy",   # 4Y
        "USSWIT5 BGN Curncy",   # 5Y
        "USSWIT10 BGN Curncy",  # 10Y
        "USSWIT20 BGN Curncy",  # 20Y
        "USSWIT30 BGN Curncy",  # 30Y
    ]

    fields = ["PX_LAST"]

    # Pull data using xbbg's bdh function
    df = blp.bdh(
        tickers=tickers,
        flds=fields,
        start_date=start_date,
        end_date=end_date
    )
    # 'df' is a multi-index DataFrame with (date) as the index and (ticker, field) as columns.
    # Drop the second level of columns ("PX_LAST"), so columns are just the tickers
    df.columns = df.columns.droplevel(level=1)

    df = df.reset_index()

    df = df.rename(columns={"index": "Dates", "date": "Dates"})

    # Reorder columns so "Dates" is first, followed by each ticker
    col_order = ["Dates"] + tickers
    df = df[col_order]

    df.to_csv(output_path, index=False)

    return df


if __name__ == "__main__":
    pull_treasury_inflation_swaps()


## Equity Spot Futures:
Manual data: "bloomberg_historical_data.parquet"

```python

def task_pull_bloomberg():
    """ """
    file_dep = [
        "./src/settings.py"
    ]
    targets = [
        INPUT_DIR / "bloomberg_historical_data.parquet"
    ]

    return {
        "actions": [
            "ipython ./src/pull_bloomberg_data.py",
        ],
        "targets": targets,
        "file_dep": file_dep,
        "clean": [],  # Don't clean these files by default. The ideas
        # is that a data pull might be expensive, so we don't want to
        # redo it unless we really mean it. So, when you run
        # doit clean, all other tasks will have their targets
        # cleaned and will thus be rerun the next time you call doit.
        # But this one wont.
        # Use doit forget --all to redo all tasks. Use doit clean
        # to clean and forget the cheaper tasks.
    }

```

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from pathlib import Path
import logging
import sys
import os
import traceback

# Add the src directory to path to load configuration settings
sys.path.insert(1, "./src")
from settings import config

# Load configuration values
USING_XBBG = config("USING_XBBG")
DATA_DIR = config("DATA_DIR")
OUTPUT_DIR = config("OUTPUT_DIR")
START_DATE = config("START_DATE")
END_DATE = config("END_DATE")
TEMP_DIR = config("TEMP_DIR")
INPUT_DIR = config("INPUT_DIR")

# Setup Bloomberg access (requires xbbg and Bloomberg Terminal)
if USING_XBBG:
    from xbbg import blp
else:
    print("Warning: xbbg not available. This script needs to be run on a machine with Bloomberg access.")


log_file_path = TEMP_DIR/f'bloomberg_data_extraction.log'
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(log_file_path),
        logging.StreamHandler(sys.stdout)
    ]
)
logger = logging.getLogger(__name__)

INDEX_CONFIG = {
    "SP": {
        "spot_ticker": "SPX Index",
        "futures_tickers": ["ES1 Index", "ES2 Index", "ES3 Index", "ES4 Index"]
    },
    "Nasdaq": {
        "spot_ticker": "NDX Index",
        "futures_tickers": ["NQ1 Index", "NQ2 Index", "NQ3 Index", "NQ4 Index"]
    },
    "DowJones": {
        "spot_ticker": "INDU Index",
        "futures_tickers": ["DM1 Index", "DM2 Index", "DM3 Index", "DM4 Index"]
    }
}

OIS_TICKERS = {
    "OIS_1W": "USSO1Z CMPN Curncy",
    "OIS_1M": "USSO1 CMPN Curncy",
    "OIS_3M": "USSOC CMPN Curncy",
    "OIS_6M": "USSOF CMPN Curncy",
    "OIS_1Y": "USSO10 CMPN Curncy"
}

def pull_spot_div_data(tickers, start_date, end_date):
    """
    Extracts spot price and dividend yield data for specified tickers from Bloomberg.

    Args:
        tickers (list): List of Bloomberg tickers (e.g., ["SPX Index"])
        start_date (str): Start date in 'YYYY-MM-DD' format
        end_date (str): End date in 'YYYY-MM-DD' format

    Returns:
        pd.DataFrame: DataFrame containing historical spot price and dividend estimates
    """
    try:
        logger.info(f"Extracting spot/dividend data for {tickers}")
        fields = ["PX_LAST", "IDX_EST_DVD_YLD", "INDX_GROSS_DAILY_DIV"]
        df = blp.bdh(tickers, fields, start_date=start_date, end_date=end_date)
        df.index = pd.to_datetime(df.index)
        return df
    except Exception as e:
        logger.error(f"Error pulling spot data for {tickers}: {e}")
        return pd.DataFrame()

def pull_futures_data(tickers, start_date, end_date):
    """
    Retrieves historical futures contract data from Bloomberg.

    Args:
        tickers (list): List of Bloomberg futures tickers (e.g., ["ES1 Index", "ES2 Index"])
        start_date (str): Start date in 'YYYY-MM-DD' format
        end_date (str): End date in 'YYYY-MM-DD' format

    Returns:
        pd.DataFrame: DataFrame with closing prices, volumes, open interest, and contract months
    """
    try:
        logger.info(f"Extracting futures data for {tickers}")
        fields = ["PX_LAST", "PX_VOLUME", "OPEN_INT", "CURRENT_CONTRACT_MONTH_YR"]
        df = blp.bdh(tickers, fields, start_date=start_date, end_date=end_date)
        df.index = pd.to_datetime(df.index)
        return df
    except Exception as e:
        logger.error(f"Error pulling futures data for {tickers}: {e}")
        return pd.DataFrame()

def pull_ois_rates(tickers, start_date, end_date):
    """
    Extracts Overnight Indexed Swap (OIS) rate data from Bloomberg.

    Args:
        tickers (list): List of OIS tickers (e.g., ["USSOC CMPN Curncy"])
        start_date (str): Start date in 'YYYY-MM-DD' format
        end_date (str): End date in 'YYYY-MM-DD' format

    Returns:
        pd.DataFrame: DataFrame containing OIS rates over time
    """
    try:
        logger.info(f"Extracting OIS rates for {tickers}")
        fields = ["PX_LAST"]
        df = blp.bdh(tickers, fields, start_date=start_date, end_date=end_date)
        df.index = pd.to_datetime(df.index)
        return df
    except Exception as e:
        logger.error(f"Error pulling OIS rates: {e}")
        return pd.DataFrame()

def main():
    """Main function to extract Bloomberg data and save it to a Parquet file."""
    if USING_XBBG:
        try:
            logger.info(f"Pulling data from {START_DATE} to {END_DATE}")

            spot_dfs, futures_dfs = [], []
            for index_name, cfg in INDEX_CONFIG.items():
                spot_df = pull_spot_div_data([cfg["spot_ticker"]], START_DATE, END_DATE)
                futures_df = pull_futures_data(cfg["futures_tickers"], START_DATE, END_DATE)
                spot_dfs.append(spot_df)
                futures_dfs.append(futures_df)

            all_spot = pd.concat(spot_dfs, axis=1) if spot_dfs else pd.DataFrame()
            all_futures = pd.concat(futures_dfs, axis=1) if futures_dfs else pd.DataFrame()

            ois_df = pull_ois_rates(list(OIS_TICKERS.values()), START_DATE, END_DATE)

            final_df = all_spot.join(all_futures, how='outer') if not all_spot.empty else all_futures
            final_df = final_df.join(ois_df, how='outer') if not final_df.empty else ois_df
            final_df.sort_index(inplace=True)
            
            INPUT_DIR.mkdir(parents=True, exist_ok=True)
            output_path = INPUT_DIR / "bloomberg_historical_data.parquet"
            final_df.to_parquet(output_path)
            logger.info(f"Final merged data saved to {output_path}")
        except Exception as e:
            logger.error(f"Error extracting Bloomberg data: {e}")
            logger.error(traceback.format_exc())
            sys.exit(1)
    else:
        logger.warning("Defaulting to cached data. Set USING_XBBG=True in settings.py to pull fresh data.")

if __name__ == "__main__":
    main()
    

## CIP:
Manual data: "CIP_2025.xlsx"

```python
def task_download_cip_data():
    """
    Download CIP_2025.xlsx from GitHub and save it to the manual data folder.
    """
    target_file = MANUAL_DATA_DIR / "CIP_2025.xlsx"

    def download():
        import requests
        url = "https://raw.githubusercontent.com/Kunj121/CIP_DATA/main/CIP_2025%20(1).xlsx"
        response = requests.get(url)
        response.raise_for_status()
        os.makedirs(MANUAL_DATA_DIR, exist_ok=True)
        with open(target_file, "wb") as f:
            f.write(response.content)
        print(f"File saved to {target_file.resolve()}")

    return {
        "actions": [download],
        "targets": [str(target_file)],
        "uptodate": [False],
        "clean": True,
    }

```

## Market Expectations
```python
def task_pull_ken_french_data():
    """Pull Data from Ken French's Website """

    return {
        "actions": [
            "ipython src/settings.py",
            "ipython src/pull_ken_french_data.py",
        ],
        "targets": [
            Path(DATA_DIR) / "6_Portfolios_2x3.xlsx",
            Path(DATA_DIR) / "25_Portfolios_5x5.xlsx",
            Path(DATA_DIR) / "100_Portfolios_10x10.xlsx",
        ],
        "file_dep": ["./src/settings.py", "./src/pull_ken_french_data.py"],
        "clean": [],  # Don't clean these files by default.
        "verbosity": 2,
    }

In [ ]:
import warnings
from pathlib import Path

import pandas as pd
import pandas_datareader.data as web

from settings import config

DATA_DIR = config("DATA_DIR")
START_DATE = config("START_DATE")
END_DATE = config("END_DATE")


def pull_ken_french_excel(
    dataset_name="Portfolios_Formed_on_INV",
    data_dir=DATA_DIR,
    log=True,
    start_date=START_DATE,
    end_date=END_DATE,
):
    """
    Pulls the Ken French portfolio data..
    
    Parameters:
    - dataset_name (str): Name of the dataset to pull.
    - data_dir (str): Directory to save the Excel file.
    - log (bool): Whether to log the path of the saved Excel file.
    - start_date (str): Start date in 'YYYY-MM-DD' format.
    - end_date (str): End date in 'YYYY-MM-DD' format.
    
    Returns:
    - Excel File: Contains date, return, and other key fields.
    """

    data_dir = Path(data_dir)
    # Suppress the specific FutureWarning about date_parser
    with warnings.catch_warnings():
        warnings.filterwarnings(
            "ignore",
            category=FutureWarning,
            message="The argument 'date_parser' is deprecated",
        )
        data = web.DataReader(
            dataset_name,
            "famafrench",
            start=start_date,
            end=end_date,
        )
        excel_path = (
            data_dir / f"{dataset_name.replace('/', '_')}.xlsx"
        )  # Ensure the name is file-path friendly

        with pd.ExcelWriter(excel_path, engine="xlsxwriter") as writer:
            # Write the description to the first sheet
            if "DESCR" in data:
                description_df = pd.DataFrame([data["DESCR"]], columns=["Description"])
                description_df.to_excel(writer, sheet_name="Description", index=False)

            # Write each table in the data to subsequent sheets
            for table_key, df in data.items():
                if table_key == "DESCR":
                    continue  # Skip the description since it's already handled
                sheet_name = str(table_key)  # Naming sheets by their table_key
                df.to_excel(
                    writer, sheet_name=sheet_name[:31]
                )  # Sheet name limited to 31 characters
    if log:
        print(f"Excel file saved to {excel_path}")
    return excel_path


def load_returns(dataset_name, weighting="value-weighted", data_dir=DATA_DIR):
    data_dir = Path(data_dir)
    excel_path = data_dir / f"{dataset_name.replace('/', '_')}.xlsx"
    if weighting == "value-weighted":
        sheet_name = "0"
    elif weighting == "equal-weighted":
        sheet_name = "1"
    else:
        raise ValueError(f"Invalid weighting: {weighting}")
    df = pd.read_excel(excel_path, sheet_name=sheet_name)
    return df


def load_sheet(dataset_name, sheet_name: str = "0", data_dir=DATA_DIR):
    """For example, for dataset_name = '6_Portfolios_2x3V', the full data
    is the description and the 10 tables of returns and properties.

    6 Portfolios 2x3
    ----------------

    
    This file was created by CMPT_ME_BEME_OP_INV_RETS using the 202412 CRSP database. It contains value- and equal-weighted returns for portfolios formed on ME and BEME. The portfolios are constructed at the end of June. BEME is book value at the last fiscal year end of the prior calendar year divided by ME at the end of December of the prior year. Annual returns are from January to December. Missing data are indicated by -99.99 or -999. The break points include utilities and include financials. The portfolios include utilities and include financials. Copyright 2024 Eugene F. Fama and Kenneth R. French

    0 : Average Value Weighted Returns -- Monthly (1129 rows x 6 cols)
    1 : Average Equal Weighted Returns -- Monthly (1129 rows x 6 cols)
    2 : Average Value Weighted Returns -- Annual (95 rows x 6 cols)
    3 : Average Equal Weighted Returns -- Annual (95 rows x 6 cols)
    4 : Number of Firms in Portfolios (1129 rows x 6 cols)
    5 : Average Market Cap (1129 rows x 6 cols)
    6 : For portfolios formed in June of year t   Value Weight Average of BE/ME Calculated for June of t to June of t+1 as:    Sum[ME(Mth) * BE(Fiscal Year t-1) / ME(Dec t-1)] / Sum[ME(Mth)]   Where Mth is a month from June of t to June of t+1   and BE(Fiscal Year t-1) is adjusted for net stock issuance to Dec t-1 (1129 rows x 6 cols)
    7 : For portfolios formed in June of year t   Value Weight Average of BE_FYt-1/ME_June t Calculated for June of t to June of t+1 as:    Sum[ME(Mth) * BE(Fiscal Year t-1) / ME(Jun t)] / Sum[ME(Mth)]   Where Mth is a month from June of t to June of t+1   and BE(Fiscal Year t-1) is adjusted for net stock issuance to Jun t (1129 rows x 6 cols)
    8 : For portfolios formed in June of year t   Value Weight Average of OP Calculated as:    Sum[ME(Mth) * OP(fiscal year t-1) / BE(fiscal year t-1)] / Sum[ME(Mth)]    Where Mth is a month from June of t to June of t+1 (727 rows x 6 cols)
    9 : For portfolios formed in June of year t   Value Weight Average of investment (rate of growth of assets) Calculated as:    Sum[ME(Mth) * Log(ASSET(t-1) / ASSET(t-2) / Sum[ME(Mth)]    Where Mth is a month from June of t to June of t+1 (727 rows x 6 cols)
    """
    
    if isinstance(sheet_name, int):
        sheet_name = str(sheet_name)

    data_dir = Path(data_dir)
    excel_path = data_dir / f"{dataset_name.replace('/', '_')}.xlsx"
    df = pd.read_excel(excel_path, sheet_name=sheet_name)
    if sheet_name == "Description":
        return df.iloc[0, 0]
    return df


def _demo():
    df = load_sheet("Portfolios_Formed_on_INV", sheet_name="0")
    df
    df_desc = load_sheet("25_Portfolios_OP_INV_5x5", sheet_name="Description")
    print(df_desc)

    ff_factors = load_sheet("F-F_Research_Data_Factors", sheet_name="0")
    ff_factors
    ff_factors_desc = load_sheet("F-F_Research_Data_Factors", sheet_name="Description")
    print(ff_factors_desc)

    ff_portfolios = load_sheet("6_Portfolios_2x3", sheet_name="0")
    ff_portfolios
    ff_portfolios_desc = load_sheet("6_Portfolios_2x3", sheet_name="Description")
    print(ff_portfolios_desc)


if __name__ == "__main__":
    _ = pull_ken_french_excel(
        dataset_name="6_Portfolios_2x3",
    )  # Save 6_Portfolios_2x3.xlsx
    _ = pull_ken_french_excel(
        dataset_name="25_Portfolios_5x5",
    )  # Save 25_Portfolios_5x5.xlsx
    _ = pull_ken_french_excel(
        dataset_name="100_Portfolios_10x10",
    )  # Save 100_Portfolios_10x10.xlsx


```python
# This is needed for CRSP value weighted index
def task_pull_CRSP_index():
    """ Pull CRSP Value-Weighted Index """
    file_dep = [
        "./src/pull_CRSP_index.py",
        ]
    file_output = [
        "crsp_value_weighted_index.csv",
        ]
    targets = [DATA_DIR / file for file in file_output]

    return {
        "actions": [
            "ipython ./src/pull_CRSP_index.py",
        ],
        "targets": targets,
        "file_dep": file_dep,
        "clean": [],  # Don't clean these files by default.
    }


In [ ]:
import wrds
import pandas as pd
from decouple import config
import os
from settings import config
from pathlib import Path


DATA_DIR = config("DATA_DIR")
START_DATE = config("START_DATE")
END_DATE = config("END_DATE")

def pull_crsp_value_weighted_index(
        data_dir=DATA_DIR,
        log=True,
        start_date=START_DATE, 
        end_date=END_DATE
):
    """
    Pulls the CRSP value-weighted index monthly returns from WRDS.
    
    Parameters:
    - start_date (str): Start date in 'YYYY-MM-DD' format.
    - end_date (str): End date in 'YYYY-MM-DD' format.
    
    Returns:
    - DataFrame: Contains date, return, and other key fields.
    """

    # Get WRDS username from environment variables or .env file
    WRDS_USERNAME = config("WRDS_USERNAME", default=os.getenv("WRDS_USERNAME"))

    if not WRDS_USERNAME:
        raise ValueError("WRDS_USERNAME is not set. Add it to your environment variables or .env file.")

    # Connect to WRDS
    db = wrds.Connection(wrds_username=WRDS_USERNAME)

    # Check available columns in crsp.msi
    table_desc = db.describe_table('crsp', 'msi')
    print("CRSP.msi Table Columns:", table_desc.columns.tolist())

    # Verify column names (adjust based on the actual table structure)
    query = f"""
        SELECT date AS date, vwretd AS value_weighted_return
        FROM crsp.msi
        WHERE date BETWEEN '{start_date}' AND '{end_date}'
        ORDER BY date;
    """

    # Execute query
    df = db.raw_sql(query, date_cols=['date'])

    # Close connection
    db.close()

    # Convert date to datetime format
    df['date'] = pd.to_datetime(df['date'])

    # Save to CSV
    csv_path = (
        data_dir / "crsp_value_weighted_index.csv"
    )

    df.to_csv(csv_path, index=False)
    # crsp_data.to_csv("crsp_value_weighted_index.csv", index=False)

    print("CRSP value-weighted index data saved to crsp_value_weighted_index.csv")
    
    if log:
        print(f"CSV file saved to {csv_path}")

    return df

if __name__ == "__main__":
    # Pull CRSP value-weighted index data
    crsp_data = pull_crsp_value_weighted_index()

## Treasury Spot
Manual Data: `OIS.xlsx`, `treasury_spot_futures.xlsx`

```python
DATA_DIR = Path(config("DATA_DIR"))
OUTPUT_DIR = Path(config("OUTPUT_DIR"))
OS_TYPE = config("OS_TYPE")
MANUAL_DATA_DIR = config("MANUAL_DATA_DIR")


# Prevent Jupyter from complaining about file path validations
environ["PYDEVD_DISABLE_FILE_VALIDATION"] = "1"
from settings import config


ois_file = f"{MANUAL_DATA_DIR}/OIS.xlsx"
data_file = f"{MANUAL_DATA_DIR}/treasury_spot_futures.xlsx"

# Comupte Series


## TIPS Treasury Arbitrage:
```python
def task_compute_tips_treasury():
    """ """
    file_dep = [
        "./src/compute_tips_treasury.py",
    ]
    targets = [
        OUTPUT_DIR / "tips_treasury_implied_rf.parquet",
    ]

    return {
        "actions": [
            "ipython ./src/compute_tips_treasury.py",
        ],
        "targets": targets,
        "file_dep": file_dep,
        "clean": [],
    }
```

In [ ]:
import numpy as np
from decouple import config
import os

DATA_DIR = config('DATA_DIR')
OUTPUT_DIR = config("OUTPUT_DIR")


# Import inflation swap data
def import_inflation_swap_data():
	# Point to the CSV file instead of an Excel file
	swaps_path = os.path.join(OUTPUT_DIR, "treasury_inflation_swaps.csv")

	# Read CSV; explicitly parse the "Dates" column as datetime
	swaps = pd.read_csv(swaps_path, parse_dates=["Dates"])

	# Create a column mapping based on the schema you provided
	column_map = {
		"Dates": "date",
		"USSWITA BGN Curncy": "inf_swap_1m",
		"USSWITC BGN Curncy": "inf_swap_3m",
		"USSWITF BGN Curncy": "inf_swap_6m",
		"USSWIT1 BGN Curncy": "inf_swap_1y",
		"USSWIT2 BGN Curncy": "inf_swap_2y",
		"USSWIT3 BGN Curncy": "inf_swap_3y",
		"USSWIT4 BGN Curncy": "inf_swap_4y",
		"USSWIT5 BGN Curncy": "inf_swap_5y",
		"USSWIT10 BGN Curncy": "inf_swap_10y",
		"USSWIT20 BGN Curncy": "inf_swap_20y",
		"USSWIT30 BGN Curncy": "inf_swap_30y"
	}

	# Rename columns using the mapping
	swaps = swaps.rename(columns=column_map)

	# Convert relevant columns to numeric and divide by 100
	inf_cols = [
		"inf_swap_1y", "inf_swap_2y", "inf_swap_3y",
		"inf_swap_4y", "inf_swap_5y", "inf_swap_10y",
		"inf_swap_20y", "inf_swap_30y"
	]
	for col in inf_cols:
		swaps[col] = pd.to_numeric(swaps[col], errors="coerce") / 100.0

	# Select only the date and inflation swap columns, in a clean order
	swaps = swaps[["date"] + inf_cols]

	return swaps

# Read in zero-coupon TIPS and Treasury yields
def import_treasury_yields():
    # Define the path to the parquet file
    nom_path = os.path.join(DATA_DIR, "fed_yield_curve.parquet")

    # Read the parquet file; date is assumed to be in the index
    nom = pd.read_parquet(nom_path)

    if not pd.api.types.is_datetime64_any_dtype(nom.index):
        nom.index = pd.to_datetime(nom.index, format="%m/%d/%Y")

    # If the index has no name or is named "Date", set it to "date"
    if nom.index.name is None or nom.index.name == "Date":
        nom.index.name = "date"

    # For each tenor (2, 5, 10, 20), compute the nominal zero-coupon yield (in basis points)
    for t in [2, 5, 10, 20]:
        col = f"SVENY{'0' + str(t) if t < 10 else str(t)}"
        nom[f"nom_zc{t}"] = 1e4 * (np.exp(nom[col] / 100) - 1)

    # Convert the date index to a column and rename it to "date" if necessary
    nom = nom.reset_index()
    nom = nom.rename(columns={'Date': 'date'})

    # Subset the DataFrame to include the date column plus the computed 'nom' columns
    nom = nom[["date"] + [col for col in nom.columns if col.startswith("nom")]]

    return nom


def import_tips_yields():
	real_path = os.path.join(DATA_DIR, "fed_tips_yield_curve.parquet")
	real = pd.read_parquet(real_path)

	if not pd.api.types.is_datetime64_any_dtype(real['Date']):
		real.rename(columns={'Date': 'date'}, inplace=True)
		real['date'] = pd.to_datetime(real['date'], format="%Y-%m-%d")

	for t in [2, 5, 10, 20]:
		col = f"TIPSY{'0' + str(t) if t < 10 else str(t)}"
		real[f"real_cc{t}"] = real[col] / 100

	real = real[["date"] + [col for col in real.columns if col.startswith("real")]]

	return real

# Merge all data, compute implied riskless rate from TIPS
def compute_tips_treasury():
	"""
	Create Constant-Maturity TIPS-Treasury Arbitrage Series and Compute Implied Risk-Free Rates

	This function merges data from three sources:
		1. TIPS yields (real rates) imported via import_tips_yields()
		2. Zero-coupon Treasury yields (nominal rates) imported via import_treasury_yields()
		3. Inflation swap data (inflation expectations) imported via import_inflation_swap_data()

	It computes for each tenor (2, 5, 10, and 20 years):
		- The TIPS-implied risk-free rate:
			tips_treas_{t}_rf = 1e4 * (exp(real_cc{t} + log(1 + inf_swap_{t}y)) - 1)
		where:
			* real_cc{t} is the continuously compounded TIPS real yield (in decimal form),
			* inf_swap_{t}y is the inflation swap rate as a decimal.
		- Arbitrage opportunities (arb_{t}) as the difference between the TIPS-implied risk-free rate
		and the nominal zero-coupon Treasury yield (nom_zc{t}). A positive arb_{t} suggests that the
		TIPS-derived rate exceeds the nominal rate, indicating a potential arbitrage opportunity.

	The final merged DataFrame, saved as a parquet file, includes:
		- date: Observation date.
		- Columns starting with "real_": TIPS real yields for each tenor (e.g., real_cc2, real_cc5, real_cc10, real_cc20)
		expressed in decimal form (e.g., 0.02 for 2%).
		- Columns starting with "nom_": Computed nominal zero-coupon Treasury yields for each tenor
		(e.g., nom_zc2, nom_zc5, nom_zc10, nom_zc20) expressed in basis points.
		- Columns starting with "tips_": TIPS-implied risk-free rates (e.g., tips_treas_2_rf, tips_treas_5_rf,
		tips_treas_10_rf, tips_treas_20_rf) expressed in basis points.
		- Columns starting with "arb_": Arbitrage measures (e.g., arb_2, arb_5, arb_10, arb_20) representing the
		difference between the TIPS-implied risk-free rate and the corresponding nominal yield (tips_treas_{t}_rf - nom_zc{t}).

	Data quality is maintained by generating missing value indicators and filtering out observations with too many missing values.
	The resulting dataset is saved as a parquet file with Snappy compression, making it ready for further analysis.
	"""
	real = import_tips_yields()
	nom = import_treasury_yields()
	swaps = import_inflation_swap_data()

	merged = pd.merge(real, nom, on="date", how="inner")
	merged = pd.merge(merged, swaps, on="date", how="inner")

	# Compute implied riskless rates from TIPS and arbitrage measures for each tenor
	missing_indicators = []
	for t in [2, 5, 10, 20]:
		merged[f"tips_treas_{t}_rf"] = 1e4 * (np.exp(merged[f"real_cc{t}"] +
														np.log(1 + merged[f"inf_swap_{t}y"])) - 1)
		merged[f"mi_{t}"] = merged[f"tips_treas_{t}_rf"].isna().astype(int)
		missing_indicators.append(f"mi_{t}")
		
		# Create new arbitrage columns with prefix "arb_"
		merged[f"arb_{t}"] = merged[f"tips_treas_{t}_rf"] - merged[f"nom_zc{t}"]

	merged["miss_count"] = merged[missing_indicators].sum(axis=1)
	merged = merged[merged["miss_count"] < 4]

	merged = merged.drop(missing_indicators + ["miss_count"], axis=1)

	

	cols_to_keep = (["date"] +
					[col for col in merged.columns if col.startswith("real_")] +
					[col for col in merged.columns if col.startswith("nom_")] +
					[col for col in merged.columns if col.startswith("tips_")] +
					[col for col in merged.columns if col.startswith("arb_")])
	merged = merged[cols_to_keep]

	output_path = os.path.join(DATA_DIR, "tips_treasury_implied_rf.parquet")
	merged.to_parquet(output_path, compression="snappy")

	print(f"Data saved to {output_path}")
	return merged 


if __name__ == "__main__":
	compute_tips_treasury()

## Equity Spot Futures:
```python

def task_process_futures_data():
    """
    Process futures data for indices after pulling the latest data.
    """
    file_dep = [
        "./src/settings.py",
        "./src/pull_bloomberg_data.py",
        "./src/futures_data_processing.py"
    ]
    targets = [
        PROCESSED_DIR / "all_indices_calendar_spreads.csv",
        PROCESSED_DIR / "INDU_calendar_spread.csv",
        PROCESSED_DIR / "SPX_calendar_spread.csv",
        PROCESSED_DIR / "NDX_calendar_spread.csv",
    ]

    return {
        "actions": [
            "python ./src/futures_data_processing.py",
        ],
        "file_dep": file_dep,
        "targets": targets,
        "clean": True,  
    }

```

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import calendar
import logging
import sys
from pathlib import Path
sys.path.insert(1, "./src")
from settings import config
from datetime import datetime
# Configuration from settings
DATA_DIR = config("DATA_DIR")
TEMP_DIR = config("TEMP_DIR")
INPUT_DIR = config("INPUT_DIR")
PROCESSED_DIR = config("PROCESSED_DIR")
DATA_MANUAL = config("MANUAL_DATA_DIR")


log_file_path = TEMP_DIR/f'futures_processing.log'
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(log_file_path),
        logging.StreamHandler(sys.stdout)
    ]
)
logger = logging.getLogger(__name__)


def get_third_friday(year, month):
    """
    Calculate the third Friday of a given month and year.
    
    Args:
        year (int): Year
        month (int): Month (1-12)
        
    Returns:
        datetime: Date object for the third Friday
    """
    # Use calendar.monthcalendar: each week is a list of ints (0 if day not in month)
    month_cal = calendar.monthcalendar(year, month)
    # The first week that has a Friday (weekday index 4)
    fridays = [week[calendar.FRIDAY] for week in month_cal if week[calendar.FRIDAY] != 0]
    if len(fridays) < 3:
        raise ValueError(f"Not enough Fridays in {year}-{month}")
    return datetime(year, month, fridays[2])  # third Friday

def parse_contract_month_year(contract_str):
    """
    Parse Bloomberg's contract month/year string (e.g., 'DEC 10') into
    a month number and a full year.
    
    Args:
        contract_str (str): Contract month/year string
        
    Returns:
        tuple: (month_num, year_full) or (None, None) if invalid.
    """
    if pd.isna(contract_str) or contract_str.strip() == '':
        return None, None
    parts = contract_str.split()
    if len(parts) != 2:
        logger.warning(f"Unexpected contract format: {contract_str}")
        return None, None
    month_abbr, year_abbr = parts
    allowed = {"MAR": 3, "JUN": 6, "SEP": 9, "DEC": 12}
    if month_abbr.upper() not in allowed:
        raise ValueError(f"Contract month {month_abbr} not in allowed set {list(allowed.keys())}")
    month_num = allowed[month_abbr.upper()]
    try:
        yr = int(year_abbr)
        year_full = 2000 + yr if yr < 50 else 1900 + yr
    except ValueError:
        logger.warning(f"Could not parse year: {year_abbr}")
        return None, None
    return month_num, year_full

def process_index_futures(data, futures_codes):
    """
    Process futures data for one index.
    
    Args:
        data (pd.DataFrame): Multi-index DataFrame with Bloomberg data (indexed by Date)
        futures_codes (list): List of futures codes (e.g., ['ES1', 'ES2', ...])
        
    Returns:
        dict: Dictionary of processed DataFrames, one for each futures code.
              Each DataFrame contains:
                  - Date
                  - Futures_Price (from PX_LAST)
                  - Volume, OpenInterest (if available)
                  - ContractSpec (raw CURRENT_CONTRACT_MONTH_YR)
                  - SettlementDate (actual settlement, 3rd Friday)
                  - TTM (time-to-maturity in days)
    """
    result_dfs = {}
    for code in futures_codes:
        logger.info(f"Processing futures data for {code} Index")
        try:
            # Extract columns for this contract:
            # Expecting columns like: (f'{code} Index', 'PX_LAST'), (f'{code} Index', 'CURRENT_CONTRACT_MONTH_YR'), etc.
            price_series = data.loc[:, (f'{code} Index', 'PX_LAST')]
            volume_series = data.loc[:, (f'{code} Index', 'PX_VOLUME')]
            oi_series = data.loc[:, (f'{code} Index', 'OPEN_INT')]
            contract_series = data.loc[:, (f'{code} Index', 'CURRENT_CONTRACT_MONTH_YR')]
            
            # Create a DataFrame for this contract; index is Date (from raw data)
            df_contract = pd.DataFrame({
                'Date': data.index,
                'Futures_Price': price_series,
                'Volume': volume_series,
                'OpenInterest': oi_series,
                'ContractSpec': contract_series
            })
            df_contract = df_contract.reset_index(drop=True)
            
            # Parse contract specification and compute settlement date
            settlement_dates = []
            for cs in df_contract['ContractSpec']:
                month_num, year_full = parse_contract_month_year(cs)
                if month_num is None or year_full is None:
                    settlement_dates.append(None)
                else:
                    settlement_dates.append(get_third_friday(year_full, month_num))
            df_contract['SettlementDate'] = pd.to_datetime(settlement_dates)
            
            # Compute TTM in days: SettlementDate - Date
            df_contract['Date'] = pd.to_datetime(df_contract['Date'])
            df_contract['TTM'] = (df_contract['SettlementDate'] - df_contract['Date']).dt.days
            
            # Drop rows with missing TTM (if settlement date couldn't be computed)
            df_contract = df_contract.dropna(subset=['TTM'])
            result_dfs[code] = df_contract
            logger.info(f"Processed {code}: {len(df_contract)} rows")
        except Exception as e:
            logger.error(f"Error processing {code}: {e}")
            continue
    return result_dfs

def merge_calendar_spreads(all_futures):
    """
    For each index, merge the processed data for the two nearest futures contracts (Term 1 and Term 2)
    on the Date field, and then combine the calendar spreads for all indices.
    
    Args:
        all_futures (dict): Dictionary keyed by index code (e.g., 'SPX', 'NDX', 'INDU') where the value is
                            another dictionary mapping futures code to its processed DataFrame.
                            
    Returns:
        pd.DataFrame: Combined calendar spread data for all indices.
    """
    combined = []
    # For each index, assume the first two codes in the list are the two nearest contracts.
    # For SPX, these would be ['ES1', 'ES2'].
    for index_code, fut_dict in all_futures.items():
        # Identify term1 and term2 codes:
        codes = list(fut_dict.keys())
        if len(codes) < 2:
            logger.warning(f"Not enough futures data for {index_code}")
            continue
        term1 = fut_dict[codes[0]].copy()
        term2 = fut_dict[codes[1]].copy()
        # Add a prefix so that we can merge and distinguish columns:
        term1 = term1.add_prefix('Term1_')
        term2 = term2.add_prefix('Term2_')
        # Rename the Date columns back to 'Date' for merging
        term1.rename(columns={'Term1_Date': 'Date'}, inplace=True)
        term2.rename(columns={'Term2_Date': 'Date'}, inplace=True)
        merged = pd.merge(term1, term2, on='Date', how='inner')
        merged['Index'] = index_code
        combined.append(merged)
        # Save each index’s calendar spread separately
        output_file = PROCESSED_DIR / f"{index_code}_calendar_spread.csv"
        merged.to_csv(output_file , index=False)
        logger.info(f"Saved calendar spread for {index_code}: {len(merged)} rows")
        logger.info(f"DataFrame merged:\n{merged.head()}")
    if combined:
        combined_df = pd.concat(combined, ignore_index=True)
        output_file = PROCESSED_DIR / "all_indices_calendar_spreads.csv"
        combined_df.to_csv(output_file, index=False)
        logger.info(f"Saved combined calendar spread data: {len(combined_df)} rows")
        logger.info(f"DataFrame combined:\n{combined_df.head()}")
        return combined_df
    else:
        logger.warning("No valid calendar spread data to combine")
        return None

def main():
    """
    Main function to process raw futures data from a parquet file.
    
    It:
      - Loads the multi-index raw data from DATA_DIR/input/bloomberg_historical_data.parquet
      - Processes each index (SPX, NDX, INDU) futures for the two nearest contracts
      - Extracts settlement dates and computes TTM
      - Merges the Term 1 and Term 2 data to form calendar spread datasets
      - Saves both individual and combined outputs for downstream spread calculations.
    """
    try:
        try:
            INPUT_FILE = INPUT_DIR / "bloomberg_historical_data.parquet"
            raw_data = pd.read_parquet(INPUT_FILE)
        except Exception as e:
            INPUT_FILE = DATA_MANUAL / "bloomberg_historical_data.parquet"
            raw_data = pd.read_parquet(INPUT_FILE)
        logger.info(f"Loading raw data from {INPUT_FILE}")
        if not isinstance(raw_data.index, pd.DatetimeIndex):
            raw_data.index = pd.to_datetime(raw_data.index)
        indices = {
            'SPX': ['ES1', 'ES2', 'ES3', 'ES4'],
            'NDX': ['NQ1', 'NQ2', 'NQ3', 'NQ4'],
            'INDU': ['DM1', 'DM2', 'DM3', 'DM4']
        }
        
        all_futures = {}
        for index_code, futures_codes in indices.items():
            logger.info(f"Processing futures for index {index_code}")
            processed = process_index_futures(raw_data, futures_codes)
            all_futures[index_code] = processed
        
        # Merge calendar spreads (using only Term 1 and Term 2)
        combined_spreads = merge_calendar_spreads(all_futures)
        logger.info("Futures data processing completed successfully")
    
    except Exception as e:
        logger.error(f"Error in main: {e}")
        raise


if __name__ == "__main__":
    main()


```python

def task_process_ois_data():
    """
    Process OIS data for 3-month rates after pulling the latest Bloomberg data.
    """
    file_dep = [
        "./src/settings.py",
        "./src/pull_bloomberg_data.py",
        "./src/OIS_data_processing.py"
    ]
    targets = [
        PROCESSED_DIR / "cleaned_ois_rates.csv"
    ]

    return {
        "actions": [
            "python ./src/OIS_data_processing.py",
        ],
        "file_dep": file_dep,
        "targets": targets,
        "clean": True,  # Add appropriate clean actions if necessary
    }

```

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import logging
import sys
import os
from pathlib import Path
sys.path.insert(1, "./src")
from settings import config

# Load configuration paths
DATA_DIR = config("DATA_DIR")
TEMP_DIR = config("TEMP_DIR")
INPUT_DIR = config("INPUT_DIR")
PROCESSED_DIR = config("PROCESSED_DIR")
DATA_MANUAL = config("MANUAL_DATA_DIR")

log_file = TEMP_DIR / f'ois_processing.log'
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(log_file),
        logging.StreamHandler(sys.stdout)
    ]
)
logger = logging.getLogger(__name__)

# Required column mapping
OIS_TENORS = {
    "OIS_3M": "USSOC CMPN Curncy",   # 3 Month OIS Rate
}

def process_ois_data(filepath: Path) -> pd.DataFrame:
    """
    Extracts, cleans, and formats only the 3-month OIS rate from Bloomberg historical dataset.

    Args:
        filepath (Path): Path to the parquet file containing multi-index Bloomberg data.

    Returns:
        pd.DataFrame: Cleaned OIS dataset containing only the 3-month OIS rate.

    Raises:
        ValueError: If the required OIS column is missing.
    """
    logger.info(f"Loading OIS data from {filepath}")

    try:
        ois_df = pd.read_parquet(filepath)
    except Exception as e:
        logger.error(f"Error reading parquet file: {e}")
        raise

    logger.info(f"Column levels: {ois_df.columns.names}")

    # Ensure required OIS column is present
    required_col = OIS_TENORS["OIS_3M"]
    if (required_col, "PX_LAST") not in ois_df.columns:
        raise ValueError(f"Missing required OIS column: {required_col}")

    # Select only the required 3-month OIS rate column
    ois_df = ois_df.loc[:, [(required_col, "PX_LAST")]]
    ois_df.columns = ["OIS_3M"]  # Rename to a clean column name

    # Convert OIS rates from percentage to decimal format (if applicable)
    logger.info("Converting OIS_3M from percentage to decimal format")
    ois_df["OIS_3M"] = ois_df["OIS_3M"] / 100

    # Drop rows with missing values
    ois_df = ois_df.dropna(subset=["OIS_3M"])

    # Save the cleaned dataset
    output_path = Path(PROCESSED_DIR) / "cleaned_ois_rates.csv"
    ois_df.to_csv(output_path, index=True)
    logger.info(f"Saved cleaned OIS rates to {output_path}")

    # Log dataset summary
    logger.info("\n========== OIS Data Summary ==========")
    logger.info(f"Shape of dataset: {ois_df.shape} (rows, columns)")
    logger.info(f"Missing values per column:\n{ois_df.isna().sum().to_string()}")
    logger.info("Descriptive statistics:\n%s", ois_df.describe().to_string())
    logger.info("First 5 rows of cleaned OIS data:\n%s", ois_df.head().to_string())

    return ois_df

def main():
    """
    Main function to process OIS rates.
    Loads Bloomberg historical data, extracts only the 3-month OIS rate,
    cleans and formats it, and saves it for further use.
    """
    INPUT_FILE = Path(INPUT_DIR) / "bloomberg_historical_data.parquet"

    if not os.path.exists(INPUT_FILE):
        logger.warning("Primary input file not found, switching to cached data")
        INPUT_FILE = Path(DATA_MANUAL) / "bloomberg_historical_data.parquet"

    try:
        process_ois_data(INPUT_FILE)
        logger.info("OIS data processing completed successfully!")
    except Exception as e:
        logger.error(f"Error processing OIS data: {e}")
        raise

if __name__ == "__main__":
    main()

```python

def task_spread_calculations():
    """
    Spread calculations from processed data
    """
    file_dep = [
        "./src/settings.py",
        "./src/pull_bloomberg_data.py",
        "./src/OIS_data_processing.py",  
        "./src/futures_data_processing.py"
    ]
    targets = [
        PROCESSED_DIR / "SPX_Forward_Rates.csv",
        PROCESSED_DIR / "NDX_Forward_Rates.csv",
        PROCESSED_DIR / "INDU_Forward_Rates.csv",
        OUTPUT_DIR / "all_indices_spread_to_2020.png",
        OUTPUT_DIR / "all_indices_spread_to_present.png"
    ]

    return {
        "actions": [
            "python ./src/Spread_calculations.py",
        ],
        "file_dep": file_dep,
        "targets": targets,
        "clean": True,  
    }
```

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from datetime import datetime
from pathlib import Path
import logging
import sys


sys.path.insert(1, "./src")
from settings import config

DATA_DIR = config("DATA_DIR")
TEMP_DIR = config("TEMP_DIR")
INPUT_DIR = config("INPUT_DIR")
PROCESSED_DIR = config("PROCESSED_DIR")
DATA_MANUAL = config("MANUAL_DATA_DIR")
OUTPUT_DIR = config("OUTPUT_DIR")

Path(OUTPUT_DIR).mkdir(exist_ok=True, parents=True)

log_file = Path(TEMP_DIR) / f"forward_rate_calculation.log"
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler(log_file),
        logging.StreamHandler(sys.stdout)
    ]
)
logger = logging.getLogger(__name__)

INDEX_CODES = ["SPX", "NDX", "INDU"]


def build_daily_dividends(index_code: str) -> pd.DataFrame:
    """
    Load daily dividends for the given index code from bloomberg_historical_data.parquet.
    Return columns: [Date, Daily_Div].
    """
    logger.info(f"[{index_code}] Building daily dividend table")

    input_file = Path(INPUT_DIR) / "bloomberg_historical_data.parquet"
    if not os.path.exists(input_file):
        logger.warning("Primary input file not found, switching to cached data")
        input_file = Path(DATA_MANUAL) / "bloomberg_historical_data.parquet"

    raw_df = pd.read_parquet(input_file)
    div_col = (f"{index_code} Index", "INDX_GROSS_DAILY_DIV")
    if div_col not in raw_df.columns:
        raise ValueError(f"Missing daily dividend column {div_col} for index={index_code}")

    div_df = raw_df.loc[:, div_col].to_frame("Daily_Div").reset_index()
    div_df.rename(columns={"index": "Date"}, inplace=True)
    div_df["Date"] = pd.to_datetime(div_df["Date"], errors="coerce")
    div_df["Daily_Div"] = div_df["Daily_Div"].fillna(0)

    # Optionally drop any row that has no valid date
    before_drop = len(div_df)
    div_df.dropna(subset=["Date"], inplace=True)
    after_drop = len(div_df)
    if after_drop < before_drop:
        logger.info(
            f"[{index_code}] Dropped {before_drop - after_drop} rows with invalid or missing date in daily_div."
        )

    div_df.sort_values("Date", inplace=True)
    div_df.reset_index(drop=True, inplace=True)

    logger.info(f"[{index_code}] daily dividends final shape: {div_df.shape}")
    logger.info(f"[{index_code}] Sample daily dividends:\n{div_df.head(10)}")
    return div_df


def barndorff_nielsen_filter(df: pd.DataFrame,
                             colname: str,
                             date_col: str = "Date",
                             window: int = 45,
                             threshold: float = 10.0) -> pd.DataFrame:
    """
    Barndorff-Nielsen outlier filter on 'colname' over ±window days.
    1) rolling median => ...
    2) abs_dev from that median
    3) rolling mean(abs_dev) => mad
    4) outlier if abs_dev/mad >= threshold => set colname_filtered=NaN
    """
    df = df.sort_values(date_col).copy()

    rolling_median = df[colname].rolling(window=window*2+1, center=True, min_periods=1).median()
    rolling_median_shifted = rolling_median.shift(1)

    df["abs_dev"] = (df[colname] - rolling_median_shifted).abs()
    rolling_mad = df["abs_dev"].rolling(window=window*2+1, center=True, min_periods=1).mean()
    rolling_mad_shifted = rolling_mad.shift(1)

    df["bad_price"] = (df["abs_dev"] / rolling_mad_shifted) >= threshold
    df.loc[df[colname].isna(), "bad_price"] = False

    # Count how many outliers
    outlier_count = df["bad_price"].sum()
    if outlier_count > 0:
        logger.info(f"Barndorff-Nielsen filter: flagged {int(outlier_count)} outliers in {colname}")

    df[f"{colname}_filtered"] = df[colname].where(~df["bad_price"], np.nan)

    df.drop(["abs_dev", "bad_price"], axis=1, inplace=True, errors="ignore")
    return df


def process_index_forward_rates(index_code: str) -> pd.DataFrame:
    """
    1) Load near/next futures for index_code from _Calendar_spread.csv
    2) Merge with single OIS_3M (as-of)
    3) Merge daily dividends, compute Div_Sum1_Comp & Div_Sum2_Comp
    4) Implied forward => cal_{index_code}_rf, OIS forward => ois_fwd_{index_code}, spread
    5) Barndorff outlier filter, then multiply spread by 100 => bps
    6) Save & return
    """
    logger.info(f"[{index_code}] Starting forward rate computation")

    fut_file = Path(PROCESSED_DIR) / f"{index_code}_Calendar_spread.csv"
    if not fut_file.exists():
        logger.error(f"[{index_code}] Missing futures file: {fut_file}")
        return pd.DataFrame()

    fut_df = pd.read_csv(fut_file)
    logger.info(f"[{index_code}] Loaded futures shape: {fut_df.shape}")


    if "Date" not in fut_df.columns:
        logger.error(f"[{index_code}] No 'Date' column in {fut_file}, aborting.")
        return pd.DataFrame()
    fut_df["Date"] = pd.to_datetime(fut_df["Date"], errors="coerce")

    before_drop = len(fut_df)
    fut_df.dropna(subset=["Date"], inplace=True)
    logger.info(f"[{index_code}] Dropped {before_drop - len(fut_df)} rows lacking a valid Date in futures.")
    fut_df["Term1_SettlementDate"] = pd.to_datetime(fut_df["Term1_SettlementDate"], errors="coerce")
    fut_df["Term2_SettlementDate"] = pd.to_datetime(fut_df["Term2_SettlementDate"], errors="coerce")

    fut_df.sort_values("Date", inplace=True)
    fut_df.reset_index(drop=True, inplace=True)

    # === Merge single OIS_3M
    ois_file = Path(PROCESSED_DIR) / "cleaned_ois_rates.csv"
    if not ois_file.exists():
        logger.error(f"[{index_code}] Missing OIS file: {ois_file}")
        return pd.DataFrame()

    ois_df = pd.read_csv(ois_file)
    if "Date" not in ois_df.columns:
        ois_df.rename(columns={"Unnamed: 0": "Date"}, inplace=True)
    ois_df["Date"] = pd.to_datetime(ois_df["Date"], errors="coerce")
    ois_df.sort_values("Date", inplace=True)

    # as-of merge
    prev_len = len(fut_df)
    merged_df = pd.merge_asof(
        fut_df, ois_df, on="Date", direction="backward"
    )
    after_len = len(merged_df)
    logger.info(f"[{index_code}] as-of merged OIS: from {prev_len} -> {after_len} rows (should be same).")

    # rename OIS_3M => 'OIS'
    if "OIS_3M" in merged_df.columns:
        merged_df.rename(columns={"OIS_3M": "OIS"}, inplace=True)
    else:
        logger.warning(f"[{index_code}] 'OIS_3M' column not found in OIS data, using default 'OIS_3M'?")

    # === Load daily dividends
    div_df = build_daily_dividends(index_code)
    # add cumsum in div_df
    div_df["CumDiv"] = div_df["Daily_Div"].cumsum()

    # merge cumsum at current date
    prev_len = len(merged_df)
    merged_df = pd.merge_asof(
        merged_df.sort_values("Date"),
        div_df[["Date", "CumDiv"]].sort_values("Date"),
        on="Date",
        direction="backward"
    )
    after_len = len(merged_df)
    logger.info(
        f"[{index_code}] as-of merged CumDiv at current date: from {prev_len} -> {after_len} rows."
    )
    merged_df.rename(columns={"CumDiv": "CumDiv_current"}, inplace=True)
    logger.info(f"[{index_code}] Sample merged rows with cumulative div:\n{merged_df.head(10)}")
    # same approach for Term1 & Term2
    t1_df = div_df.rename(columns={"Date": "Term1_SettlementDate", "CumDiv": "CumDiv_Term1"})
    prev_len = len(merged_df)
    merged_df = pd.merge_asof(
        merged_df.sort_values("Term1_SettlementDate"),
        t1_df.sort_values("Term1_SettlementDate"),
        on="Term1_SettlementDate",
        direction="backward"
    )
    after_len = len(merged_df)
    logger.info(
        f"[{index_code}] as-of merged CumDiv for Term1: from {prev_len} -> {after_len} rows."
    )

    t2_df = div_df.rename(columns={"Date": "Term2_SettlementDate", "CumDiv": "CumDiv_Term2"})
    prev_len = len(merged_df)
    merged_df = pd.merge_asof(
        merged_df.sort_values("Term2_SettlementDate"),
        t2_df.sort_values("Term2_SettlementDate"),
        on="Term2_SettlementDate",
        direction="backward"
    )
    after_len = len(merged_df)
    logger.info(
        f"[{index_code}] as-of merged CumDiv for Term2: from {prev_len} -> {after_len} rows."
    )

    # compute Div_Sum1 & Div_Sum2
    merged_df["Div_Sum1"] = merged_df["CumDiv_Term1"] - merged_df["CumDiv_current"]
    merged_df["Div_Sum2"] = merged_df["CumDiv_Term2"] - merged_df["CumDiv_current"]

    # Handle missing TTM or price
    # If TTM is missing, can't compute rates => drop
    before_drop = len(merged_df)
    merged_df.dropna(subset=["Term1_TTM", "Term2_TTM", "Term1_Futures_Price", "Term2_Futures_Price"], inplace=True)
    logger.info(
        f"[{index_code}] Dropped {before_drop - len(merged_df)} rows missing TTM or Futures_Price."
    )

    # 4) Compounding
    ttm1 = "Term1_TTM"
    ttm2 = "Term2_TTM"
    merged_df["Div_Sum1_Comp"] = merged_df["Div_Sum1"] * (
        ((merged_df[ttm1] / 2.0) / 360.0) * merged_df["OIS"] + 1.0
    )
    merged_df["Div_Sum2_Comp"] = merged_df["Div_Sum2"] * (
        ((merged_df[ttm2] / 2.0) / 360.0) * merged_df["OIS"] + 1.0
    )

    # Implied Forward
    fp1 = "Term1_Futures_Price"
    fp2 = "Term2_Futures_Price"
    merged_df["implied_forward_raw"] = (
        (merged_df[fp2] + merged_df["Div_Sum2_Comp"]) /
        (merged_df[fp1] + merged_df["Div_Sum1_Comp"])
        - 1.0
    )

    dt = merged_df[ttm2] - merged_df[ttm1]
    merged_df[f"cal_{index_code}_rf"] = np.where(
        dt > 0,
        100.0 * merged_df["implied_forward_raw"] * (360.0 / dt),
        np.nan
    )

    # OIS-implied forward
    merged_df["ois_fwd_raw"] = (
        (1.0 + merged_df["OIS"] * merged_df[ttm2] / 360.0) /
        (1.0 + merged_df["OIS"] * merged_df[ttm1] / 360.0)
        - 1.0
    )
    merged_df[f"ois_fwd_{index_code}"] = np.where(
        dt > 0,
        merged_df["ois_fwd_raw"] * (360.0 / dt) * 100.0,
        np.nan
    )

    # Spread
    spread_col = f"spread_{index_code}"
    merged_df[spread_col] = merged_df[f"cal_{index_code}_rf"] - merged_df[f"ois_fwd_{index_code}"]
    # 8) BN outlier filter
    merged_df = barndorff_nielsen_filter(merged_df, spread_col, date_col="Date", window=45, threshold=10)
    # If outlier => set cal_rf & spread to NaN
    out_mask = merged_df[f"{spread_col}_filtered"].isna()
    outliers_count = out_mask.sum()
    if outliers_count > 0:
        logger.info(f"[{index_code}] Setting {outliers_count} outliers to NaN for cal_{index_code}_rf & {spread_col}")
    merged_df.loc[out_mask, f"cal_{index_code}_rf"] = np.nan
    merged_df.loc[out_mask, spread_col] = np.nan

    # Multiply spread by 100 => bps

    merged_df[spread_col] = merged_df[spread_col] * 100.0
    merged_df.set_index("Date", inplace=True)
    out_file = Path(PROCESSED_DIR) / f"{index_code}_Forward_Rates.csv"
    merged_df.to_csv(out_file)
    logger.info(f"[{index_code}] Final forward rates shape: {merged_df.shape}, saved to {out_file}")
    logger.info(
        f"[{index_code}] Sample final rows:\n"
        + merged_df[[f"cal_{index_code}_rf", f"ois_fwd_{index_code}", spread_col]].tail(5).to_string()
    )

    return merged_df


def plot_all_indices(results: dict, keep_dates: bool = True):
    """
    Generate two plots:
    1) From START_DATE to 01/01/2020
    2) From START_DATE to END_DATE

    If keep_dates=True, reindex all series to the union of dates in results.
    """
    START_DATE = pd.to_datetime(config("START_DATE"))
    END_DATE = pd.to_datetime(config("END_DATE"))
    MID_DATE = pd.to_datetime("2020-01-01")

    def _plot(date_range, filename_suffix):
        plt.figure(figsize=(12, 7))

        date_index = None
        if keep_dates:
            all_dates = set()
            for df in results.values():
                if df is not None and not df.empty:
                    all_dates.update(df.index)
            date_index = pd.to_datetime(sorted(all_dates))

        colors = {"SPX": "blue", "NDX": "green", "INDU": "red"}

        for idx, df in results.items():
            if df is not None and not df.empty:
                spread_col = f"spread_{idx}"
                df_plot = df.reindex(date_index).ffill() if keep_dates and date_index is not None else df
                df_plot = df_plot.loc[(df_plot.index >= START_DATE) & (df_plot.index <= date_range)]
                
                plt.plot(df_plot.index, df_plot[spread_col], color=colors.get(idx, "black"), alpha=0.8, label=f"{idx} Spread (bps)")

        plt.axhline(0, color="k", linestyle="--", alpha=0.7)
        plt.title(f"Implied Forward Spread Across Indices (bps)\n[{START_DATE.date()} to {date_range.date()}]")
        plt.xlabel("Date")
        plt.ylabel("Spread (bps)")
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        ax = plt.gca()
        ax.xaxis.set_major_locator(mdates.YearLocator())
        ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y"))

        out_png = Path(OUTPUT_DIR) / f"all_indices_spread_{filename_suffix}.png"
        plt.savefig(out_png, dpi=300)
        logger.info(f"Saved plot to {out_png}")
        plt.close()

    # Generate two plots with different date ranges
    _plot(MID_DATE, "to_2020")
    _plot(END_DATE, "to_present")




def main():
    logger.info("== Starting forward rate calculations with compounding dividends, single OIS, BN outlier filter ==")
    results = {}
    for idx in INDEX_CODES:
        df_res = process_index_forward_rates(idx)
        results[idx] = df_res
    plot_all_indices(results, keep_dates=True)

    logger.info("All computations completed successfully.")


if __name__ == "__main__":
    main()


## CIP

```python
def task_clean_data():
    """Run the CIP data cleaning script."""
    tidy_data_file = DATA_DIR / "tidy_data.csv"
    return {
        "actions": ["ipython ./src/clean_data.py"],
        "file_dep": [str(MANUAL_DATA_DIR / "CIP_2025.xlsx")],
        "targets": [str(tidy_data_file)],
        "clean": True,
    }

In [ ]:
# double check this works
from pathlib import Path
try:
    from pull_bloomberg_cip_data import *
    import pull_bloomberg_cip_data as pull_bloomberg_cip_data
except ModuleNotFoundError:
    from src.pull_bloomberg_cip_data import *
    import src.pull_bloomberg_cip_data as pull_bloomberg_cip_data

try:
    from settings import config
except ModuleNotFoundError:
    from src.settings import config

DATA_DIR = Path(config("DATA_DIR"))  # Should point to '_data'

df = pull_bloomberg_cip_data.load_raw('2025-03-01')
output_file = DATA_DIR / "tidy_data.csv"

df.to_csv(output_file, index=False)

print(f"Cleaned data saved to {output_file}")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import requests
from io import BytesIO
import sys
import os

# Ensure the root directory (CIP/) is in sys.path
sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__), "..")))

try:
    import src.settings as settings  # Try to import normally
except ModuleNotFoundError:
    import settings as settings # Fallback if src.settings isn't found


BLOOMBERG = settings.BLOOMBERG

def download():
    target_file = "./data_manual/CIP_2025.xlsx"
    import requests, os
    url = "https://raw.githubusercontent.com/Kunj121/CIP_DATA/main/CIP_2025%20(1).xlsx"
    response = requests.get(url)
    response.raise_for_status()  # Raise an error on bad responses
    # Ensure the data_manual folder exists.
    os.makedirs(os.path.dirname(target_file), exist_ok=True)
    # Write the file to the target path.
    with open(target_file, "wb") as f:
        f.write(response.content)
    df =  pd.read_excel(target_file)
    return df




def fetch_bloomberg_historical_data(start_date, end_date):
    """
    Fetch historical data from Bloomberg using xbbg for predefined sets of tickers,
    clean up the data, and merge into a single DataFrame similar to the existing process.

    Parameters
    ----------
    start_date : str, optional
        Start date in 'YYYY-MM-DD' format, defaults to "2010-01-01"
    end_date : str, optional
        End date in 'YYYY-MM-DD' format, defaults to "2025-12-31"

    Returns
    -------
    pandas.DataFrame
        A merged DataFrame containing all the processed historical data
        (spot rates, swap rates, interest rates) for AUD, CAD, CHF, EUR,
        GBP, JPY, NZD, and SEK (with USD as reference).
    """
    from xbbg import blp
    start_date = "2010-01-01"
    end_date = "2025-12-31"

    # Tickers for Spot Rates
    interest_rates = [
        "ADSOC CMPN Curncy",  # AUD
        "CDSOC CMPN Curncy",  # CAD
        "SFSNTC CMPN Curncy", # CHF
        "EUSWEC CMPN Curncy", # EUR
        "BPSWSC CMPN Curncy", # GBP
        "JYSOC CMPN Curncy",  # JPY
        "NDSOC CMPN Curncy",  # NZD
        "SKSWTNC BGN Curncy", # SEK
        "USSOC CMPN Curncy",  # USD
    ]

    # Tickers for 3M interest rates (overnight or 3M LIBOR, depending on the data vendor)
    forward_rates = [
        "AUD3M CMPN Curncy",
        "CAD3M CMPN Curncy",
        "CHF3M CMPN Curncy",
        "EUR3M CMPN Curncy",
        "GBP3M CMPN Curncy",
        "JPY3M CMPN Curncy",
        "NZD3M CMPN Curncy",
        "SEK3M CMPN Curncy"
    ]


    # Tickers for 3M swap rates (or 3M forward quotes)
    spot_rates = [
        "AUD CMPN Curncy",
        "CAD CMPN Curncy",
        "CHF CMPN Curncy",
        "EUR CMPN Curncy",
        "GBP CMPN Curncy",
        "JPY CMPN Curncy",
        "NZD CMPN Curncy",
        "SEK CMPN Curncy"
    ]

    # Mapping from Bloomberg columns to simpler names
   
    IR_mapping = {
        "ADSOC CMPN Curncy_PX_LAST": "AUD_IR",
        "CDSOC CMPN Curncy_PX_LAST": "CAD_IR",
        "SFSNTC CMPN Curncy_PX_LAST": "CHF_IR",
        "EUSWEC CMPN Curncy_PX_LAST": "EUR_IR",
        "BPSWSC CMPN Curncy_PX_LAST": "GBP_IR",
        "JYSOC CMPN Curncy_PX_LAST": "JPY_IR",
        "NDSOC CMPN Curncy_PX_LAST": "NZD_IR",
        "SKSWTNC BGN Curncy_PX_LAST": "SEK_IR",
        "USSOC CMPN Curncy_PX_LAST": "USD_IR"
    }

    forward_mapping = {
        "AUD3M CMPN Curncy_PX_LAST": "AUD_CURNCY3M",
        "CAD3M CMPN Curncy_PX_LAST": "CAD_CURNCY3M",
        "CHF3M CMPN Curncy_PX_LAST": "CHF_CURNCY3M",
        "EUR3M CMPN Curncy_PX_LAST": "EUR_CURNCY3M",
        "GBP3M CMPN Curncy_PX_LAST": "GBP_CURNCY3M",
        "JPY3M CMPN Curncy_PX_LAST": "JPY_CURNCY3M",
        "NZD3M CMPN Curncy_PX_LAST": "NZD_CURNCY3M",
        "SEK3M CMPN Curncy_PX_LAST": "SEK_CURNCY3M"
    }

    spot_mapping = {
        "AUD CMPN CURNCY_PX_LAST": "AUD_CURNCY",
        "CAD CMPN CURNCY_PX_LAST": "CAD_CURNCY",
        "CHF CMPN CURNCY_PX_LAST": "CHF_CURNCY",
        "EUR CMPN CURNCY_PX_LAST": "EUR_CURNCY",
        "GBP CMPN CURNCY_PX_LAST": "GBP_CURNCY",
        "JPY CMPN CURNCY_PX_LAST": "JPY_CURNCY",
        "NZD CMPN CURNCY_PX_LAST": "NZD_CURNCY",
        "SEK CMPN CURNCY_PX_LAST": "SEK_CURNCY"
    }

    fields = ["PX_LAST"]

    # Helper to flatten the multi-index
    def process_df(df, column_mapping):
        if not df.empty:
            df.columns = [f"{t[0]}_{t[1]}" for t in df.columns]
            df.rename(columns=column_mapping, inplace=True)
            df.set_index('date', inplace=True)
        return df

    # Pull each set of tickers
    interest_rates_df = process_df(
        blp.bdh(
            tickers=interest_rates,
            flds=fields,
            start_date=start_date,
            end_date=end_date,
        ),
        IR_mapping
    )

    forward_rates_df = process_df(
        blp.bdh(
            tickers=forward_rates,
            flds=fields,
            start_date=start_date,
            end_date=end_date,
        ),
        forward_mapping
    )

    exchange_rates_df = process_df(
        blp.bdh(
            tickers=spot_rates,
            flds=fields,
            start_date=start_date,
            end_date=end_date,
        ),
        spot_mapping
    )

    # For demonstration, we replicate the "3M forward" concept by appending
    # the spot data (currency_df) to the swap rates (swap_df).
    # In a real scenario, you might need the forward points separately
    # to add to / subtract from spot. This code treats 'swap_df' as 3M rates.

    # We keep the original spot rates in swap_df as well for reference:
    cols = ["AUD", "CAD", "CHF", "EUR", "GBP", "JPY", "NZD", "SEK"]
    cols_IR = ["AUD", "CAD", "CHF", "EUR", "GBP", "JPY", "NZD", "SEK", "USD"]
    exchange_rates_df.columns = cols
    forward_rates_df.columns = cols
    interest_rates_df.columns = cols_IR

    # Convert certain currencies to reciprocals
    # The forward df is actually forward points, so we need to make this into forward rates.
    forward_rates_df[[c for c in cols if c != 'JPY']] /= 10000
    forward_rates_df['JPY'] /= 100
    forward_rates_df = exchange_rates_df + forward_rates_df

    exchange_rates_df.columns = [name+"_CURNCY" for name in exchange_rates_df.columns]
    forward_rates.columns = [name+"_CURNCY3M" for name in forward_rates.columns]
    interest_rates.columns = [name+"_IR" for name in interest_rates.columns]

    # Merge all
    df_merged = (
        exchange_rates_df
        .merge(forward_rates_df, left_index=True, right_index=True, how='inner')
        .merge(interest_rates_df, left_index=True, right_index=True, how='inner')
    )

    return df_merged


def plot_cip(end ='2025-03-01'):
    """
    Reads data from Excel if excel=True, otherwise fetch from Bloomberg using xbbg.

    After retrieving data, calculates a CIP measure and cleans outliers.
    Finally, plots the CIP spreads for a 2010-2019 subset and the full range.

    Parameters
    ----------
    start : str, optional
        Start date in 'YYYY-MM-DD' format, used if excel=False
    end : str, optional
        End date in 'YYYY-MM-DD' format, used if excel=False
    excel : bool, optional
        If True, read from a local Excel file. If False, use Bloomberg xbbg.

    Returns
    -------
    df_merged : pandas.DataFrame
        Final cleaned DataFrame with CIP spreads and underlying data.
    """
    start = '2010-01-01'
    if BLOOMBERG == False:
        possible_paths = [
            "./data_manual/CIP_2025.xlsx",
            "../data_manual/CIP_2025.xlsx",
        ]

        data = None
        for filepath in possible_paths:
            if os.path.exists(filepath):
                try:
                    data = pd.read_excel(filepath, sheet_name=None)
                    break
                except Exception as e:
                    print(f"Error loading {filepath}: {e}")
            else:
                pass
        if data is None:
            raise FileNotFoundError("Could not find or load the CIP_2025.xlsx file in any of the expected locations")

        df_spot = data["Spot"]
        exchange_rates = df_spot.set_index("Date")

        df_forward = data["Forward"]
        forward_rates = df_forward.set_index("Date")

        df_ir = data["OIS"]
        interest_rates = df_ir.set_index("Date")
        # Standard columns
        cols = ["AUD", "CAD", "CHF", "EUR", "GBP", "JPY", "NZD", "SEK"]
        exchange_rates.columns = cols
        forward_rates.columns = cols

        # Convert forward points to forward rates
        # Non-JPY: forward points are per 10,000; JPY: per 100
        forward_rates[[c for c in cols if c != 'JPY']] /= 10000
        forward_rates['JPY'] /= 100
        forward_rates = exchange_rates + forward_rates

        # Rename to keep track
        exchange_rates.columns = [f"{name}_CURNCY" for name in exchange_rates.columns]
        forward_rates.columns = [f"{name}_CURNCY3M" for name in forward_rates.columns]
        interest_rates.columns = [f"{name}_IR" for name in interest_rates.columns]

        # Merge
        df_merged = (
            exchange_rates
            .merge(forward_rates, left_index=True, right_index=True, how='inner')
            .merge(interest_rates, left_index=True, right_index=True, how='inner')
        )

        # Convert to reciprocal for these currencies
        reciprocal_currencies = ['EUR', 'GBP', 'AUD', 'NZD']
        for ccy in reciprocal_currencies:
            df_merged[f"{ccy}_CURNCY"] = 1.0 / df_merged[f"{ccy}_CURNCY"]
            df_merged[f"{ccy}_CURNCY3M"] = 1.0 / df_merged[f"{ccy}_CURNCY3M"]

    else:
        # 2) Pull from Bloomberg
        df_merged = fetch_bloomberg_historical_data(start, end)

    # List of all the core currencies
    currencies = ['AUD', 'CAD', 'CHF', 'EUR', 'GBP', 'JPY', 'NZD', 'SEK']

    ######################################
    # Compute the log CIP basis in basis points
    ######################################
    for ccy in currencies:
        fwd_col    = f'{ccy}_CURNCY3M'
        spot_col   = f'{ccy}_CURNCY'
        ir_col     = f'{ccy}_IR'
        usd_ir_col = 'USD_IR'  # The US interest rate column

        # CIP in log terms (bps) = 100*100 x [ domestic_i - (logF - logS)*(360/90) - foreign_i ]
        cip_col = f'CIP_{ccy}_ln'
        df_merged[cip_col] = 100*100 * (
            (df_merged[ir_col] / 100.0)               # domestic interest rate
            - (360.0 / 90.0) * (
                np.log(df_merged[fwd_col]) - np.log(df_merged[spot_col])
            )
            - (df_merged[usd_ir_col] / 100.0)         # foreign interest rate (USD)
        )

    ######################################
    # Rolling outlier cleanup (45-day window)
    ######################################
    window_size = 45
    for ccy in currencies:
        cip_col = f'CIP_{ccy}_ln'
        if cip_col not in df_merged.columns:
            continue

        # Rolling median over 45 days
        rolling_median = df_merged[cip_col].rolling(window_size).median()

        # Absolute deviation from median
        abs_dev = (df_merged[cip_col] - rolling_median).abs()

        # Rolling mean of abs_dev (proxy for MAD)
        rolling_mad = abs_dev.rolling(window_size).mean()

        # Mark outliers (abs_dev / mad >= 10) and replace with NaN
        outlier_mask = (abs_dev / rolling_mad) >= 10
        df_merged.loc[outlier_mask, cip_col] = np.nan

    # Create a separate DataFrame for the CIP columns
    cip_cols = [f'CIP_{c}_ln' for c in currencies if f'CIP_{c}_ln' in df_merged.columns]
    spreads = df_merged[cip_cols].copy()

    # Shorten column names for plotting
    spreads.columns = [c[4:7] for c in spreads.columns]  # e.g., CIP_AUD_ln -> AUD





    def plot_spreads(spreads_df, yr):
        """
        Plots the CIP spreads in basis points.
        Saves both a PDF and PNG with the specified yr suffix in the filename.
        """
        fig, ax = plt.subplots(figsize=(13, 8), dpi=300)

        # Plot each column in the DataFrame
        for column in spreads_df.columns:
            ax.plot(spreads_df.index, spreads_df[column], label=column, linewidth=1, antialiased=True)

        ax.set_xlabel("Dates", fontsize=14)
        ax.set_ylabel("Arbitrage Spread (bps)", fontsize=14)

        # Format the x-axis for years
        ax.xaxis.set_major_locator(mdates.YearLocator(2))
        ax.xaxis.set_minor_locator(mdates.YearLocator(1))
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
        plt.xticks(rotation=0)

        # Horizontal grid lines only
        ax.yaxis.grid(True, linestyle="--", alpha=0.5)
        ax.xaxis.grid(False)

        # Hard limit the y-axis
        ax.set_ylim([-50, 210])

        # Legend below the plot
        ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.2), ncol=4, fontsize=12, frameon=True)

        # Remove top and right spines
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)

        plt.tight_layout(rect=[0, 0.15, 1, 1])  # ensure legend fits

        plt.savefig(f"spread_plot_{yr}.pdf", format="pdf", bbox_inches='tight')
        plt.savefig(f"spread_plot_{yr}.png", dpi=300, bbox_inches='tight')

    # Plot from start to 2019, and the full range
    if isinstance(df_merged.index, pd.DatetimeIndex):
        plot_spreads(spreads.loc[:end], 'rep')




def load_raw(end ='2025-03-01', plot = False):
    """
    Reads data from Excel if excel=True, otherwise fetch from Bloomberg using xbbg.

    After retrieving data, calculates a CIP measure and cleans outliers.
    Finally, plots the CIP spreads for a 2010-2019 subset and the full range.

    Parameters
    ----------
    start : str, optional
        Start date in 'YYYY-MM-DD' format, used if excel=False
    end : str, optional
        End date in 'YYYY-MM-DD' format, used if excel=False
    excel : bool, optional
        If True, read from a local Excel file. If False, use Bloomberg xbbg.

    Returns
    -------
    df_merged : pandas.DataFrame
        Final cleaned DataFrame with CIP spreads and underlying data.
    """

    start = '2010-01-01'
    if BLOOMBERG == False:
        possible_paths = [
            "./data_manual/CIP_2025.xlsx",
            "../data_manual/CIP_2025.xlsx",
        ]

        data = None
        for filepath in possible_paths:
            if os.path.exists(filepath):
                try:
                    data = pd.read_excel(filepath, sheet_name=None)
                    break
                except Exception as e:
                    print(f"Error loading {filepath}: {e}")
            else:
                pass
        if data is None:
            download()
        data = pd.read_excel(filepath, sheet_name=None, parse_dates=['Date'])

        df_spot = data["Spot"]
        exchange_rates = df_spot.set_index("Date")

        df_forward = data["Forward"]
        forward_rates = df_forward.set_index("Date")

        df_ir = data["OIS"]
        interest_rates = df_ir.set_index("Date")

        # Standard columns
        cols = ["AUD", "CAD", "CHF", "EUR", "GBP", "JPY", "NZD", "SEK"]
        exchange_rates.columns = cols
        forward_rates.columns = cols

        # Convert forward points to forward rates
        # Non-JPY: forward points are per 10,000; JPY: per 100
        forward_rates[[c for c in cols if c != 'JPY']] /= 10000
        forward_rates['JPY'] /= 100
        forward_rates = exchange_rates + forward_rates

        # Rename to keep track
        exchange_rates.columns = [f"{name}_CURNCY" for name in exchange_rates.columns]
        forward_rates.columns = [f"{name}_CURNCY3M" for name in forward_rates.columns]
        interest_rates.columns = [f"{name}_IR" for name in interest_rates.columns]

        # Merge
        df_merged = (
            exchange_rates
            .merge(forward_rates, left_index=True, right_index=True, how='inner')
            .merge(interest_rates, left_index=True, right_index=True, how='inner')
        )

        # Convert to reciprocal for these currencies
        reciprocal_currencies = ['EUR', 'GBP', 'AUD', 'NZD']
        for ccy in reciprocal_currencies:
            df_merged[f"{ccy}_CURNCY"] = 1.0 / df_merged[f"{ccy}_CURNCY"]
            df_merged[f"{ccy}_CURNCY3M"] = 1.0 / df_merged[f"{ccy}_CURNCY3M"]


    else:
        # 2) Pull from Bloomberg
        df_merged = fetch_bloomberg_historical_data(start, end)


    return df_merged.loc[:end]

def compute_cip(end = '2020-01-01'):
    df_merged = load_raw(end = end)

    # List of all the core currencies
    currencies = ['AUD', 'CAD', 'CHF', 'EUR', 'GBP', 'JPY', 'NZD', 'SEK']

    ######################################
    # Compute the log CIP basis in basis points
    ######################################
    for ccy in currencies:
        fwd_col = f'{ccy}_CURNCY3M'
        spot_col = f'{ccy}_CURNCY'
        ir_col = f'{ccy}_IR'
        usd_ir_col = 'USD_IR'  # The US interest rate column

        # CIP in log terms (bps) = 100*100 x [ domestic_i - (logF - logS)*(360/90) - foreign_i ]
        cip_col = f'CIP_{ccy}_ln'
        df_merged[cip_col] = 100 * 100 * (
                (df_merged[ir_col] / 100.0)  # domestic interest rate
                - (360.0 / 90.0) * (
                        np.log(df_merged[fwd_col]) - np.log(df_merged[spot_col])
                )
                - (df_merged[usd_ir_col] / 100.0)  # foreign interest rate (USD)
        )

    ######################################
    # Rolling outlier cleanup (45-day window)
    ######################################
    window_size = 45
    for ccy in currencies:
        cip_col = f'CIP_{ccy}_ln'
        if cip_col not in df_merged.columns:
            continue

        # Rolling median over 45 days
        rolling_median = df_merged[cip_col].rolling(window_size).median()

        # Absolute deviation from median
        abs_dev = (df_merged[cip_col] - rolling_median).abs()

        # Rolling mean of abs_dev (proxy for MAD)
        rolling_mad = abs_dev.rolling(window_size).mean()

        # Mark outliers (abs_dev / mad >= 10) and replace with NaN
        outlier_mask = (abs_dev / rolling_mad) >= 10
        df_merged.loc[outlier_mask, cip_col] = np.nan

    # Create a separate DataFrame for the CIP columns
    cip_cols = [f'CIP_{c}_ln' for c in currencies if f'CIP_{c}_ln' in df_merged.columns]
    spreads = df_merged[cip_cols].copy()

    # Shorten column names for plotting
    spreads.columns = [c[4:7] for c in spreads.columns]  # e.g., CIP_AUD_ln -> AUD

    return df_merged.iloc[:, -8:]


def load_raw_pieces(end ='2025-03-01',excel=False, plot = False):
    """
    Reads data from Excel if excel=True, otherwise fetch from Bloomberg using xbbg.

    After retrieving data, calculates a CIP measure and cleans outliers.
    Finally, plots the CIP spreads for a 2010-2019 subset and the full range.

    Parameters
    ----------
    start : str, optional
        Start date in 'YYYY-MM-DD' format, used if excel=False
    end : str, optional
        End date in 'YYYY-MM-DD' format, used if excel=False
    excel : bool, optional
        If True, read from a local Excel file. If False, use Bloomberg xbbg.

    Returns
    -------
    df_merged : pandas.DataFrame
        Final cleaned DataFrame with CIP spreads and underlying data.
    """
    start = '2010-01-01'
    if BLOOMBERG == False:
        possible_paths = [
            "./data_manual/CIP_2025.xlsx",
            "../data_manual/CIP_2025.xlsx",
        ]

        data = None
        for filepath in possible_paths:
            if os.path.exists(filepath):
                try:
                    data = pd.read_excel(filepath, sheet_name=None)
                    break
                except Exception as e:
                    print(f"Error loading {filepath}: {e}")
            else:
                pass
        df_spot = data["Spot"]
        exchange_rates = df_spot.set_index("Date")

        df_forward = data["Forward"]
        forward_rates = df_forward.set_index("Date")

        df_ir = data["OIS"]
        interest_rates = df_ir.set_index("Date")

        # Standard columns
        cols = ["AUD", "CAD", "CHF", "EUR", "GBP", "JPY", "NZD", "SEK"]
        exchange_rates.columns = cols
        forward_rates.columns = cols

        # Convert forward points to forward rates
        # Non-JPY: forward points are per 10,000; JPY: per 100
        forward_rates[[c for c in cols if c != 'JPY']] /= 10000
        forward_rates['JPY'] /= 100
        forward_rates = exchange_rates + forward_rates

        # Rename to keep track
        exchange_rates.columns = [f"{name}_CURNCY" for name in exchange_rates.columns]
        forward_rates.columns = [f"{name}_CURNCY3M" for name in forward_rates.columns]
        interest_rates.columns = [f"{name}_IR" for name in interest_rates.columns]

        # Merge
        df_merged = (
            exchange_rates
            .merge(forward_rates, left_index=True, right_index=True, how='inner')
            .merge(interest_rates, left_index=True, right_index=True, how='inner')
        )

        # Convert to reciprocal for these currencies
        reciprocal_currencies = ['EUR', 'GBP', 'AUD', 'NZD']
        for ccy in reciprocal_currencies:
            df_merged[f"{ccy}_CURNCY"] = 1.0 / df_merged[f"{ccy}_CURNCY"]
            df_merged[f"{ccy}_CURNCY3M"] = 1.0 / df_merged[f"{ccy}_CURNCY3M"]

    else:
        # 2) Pull from Bloomberg
        df_merged = fetch_bloomberg_historical_data(start, end)


    exchange_rates_df = df_merged.iloc[:,:8]
    forward_rates_df = df_merged.iloc[:, 8:16]
    interest_rates_df = df_merged.iloc[:, -9:]

    return exchange_rates_df, forward_rates_df, interest_rates_df


```python

def task_summary_stats():
    """Generate summary statistics and save them as HTML files."""
    def generate_summary():
        # Ensure we correctly import the required functions
        from src.directory_functions import save_cip_statistics_as_html
        from src.pull_bloomberg_cip_data import compute_cip
        from src.cip_analysis import compute_cip_statistics

        # Step 1: Compute CIP data
        cip_data = compute_cip()

        # Step 2: Compute statistics
        stats_dict = compute_cip_statistics(cip_data)

        # Debugging: Check if 'overall_statistics' exists
        if "overall_statistics" not in stats_dict:
            raise KeyError("Error: 'overall_statistics' key is missing from stats_dict!")

        # Step 3: Save statistics as HTML
        save_cip_statistics_as_html(stats_dict)

    return {
        "actions": [generate_summary],
        "file_dep": ["./src/pull_bloomberg_cip_data.py", "./src/cip_analysis.py"],
        "targets": [
            str(OUTPUT_DIR / "cip_summary_overall.html"),
            str(OUTPUT_DIR / "cip_correlation_matrix.html"),
            str(OUTPUT_DIR / "cip_annual_statistics.html"),
        ],
        "clean": True,
    }

In [ ]:

def save_cip_statistics_as_html(stats_dict):
    output_dir = os.path.join(OUTPUT_DIR, "html_files")
    """Save CIP statistics as HTML files."""
    os.makedirs(output_dir, exist_ok=True)

    overall_html = os.path.join(output_dir, "cip_summary_overall.html")
    corr_html = os.path.join(output_dir, "cip_correlation_matrix.html")
    annual_html = os.path.join(output_dir, "cip_annual_statistics.html")

    stats_dict["overall_statistics"].to_html(overall_html)
    stats_dict["correlation_matrix"].to_html(corr_html)
    stats_dict["annual_statistics"].to_html(annual_html)

    return overall_html, corr_html, annual_html

def compute_cip(end = '2020-01-01'):
    df_merged = load_raw(end = end)

    # List of all the core currencies
    currencies = ['AUD', 'CAD', 'CHF', 'EUR', 'GBP', 'JPY', 'NZD', 'SEK']

    ######################################
    # Compute the log CIP basis in basis points
    ######################################
    for ccy in currencies:
        fwd_col = f'{ccy}_CURNCY3M'
        spot_col = f'{ccy}_CURNCY'
        ir_col = f'{ccy}_IR'
        usd_ir_col = 'USD_IR'  # The US interest rate column

        # CIP in log terms (bps) = 100*100 x [ domestic_i - (logF - logS)*(360/90) - foreign_i ]
        cip_col = f'CIP_{ccy}_ln'
        df_merged[cip_col] = 100 * 100 * (
                (df_merged[ir_col] / 100.0)  # domestic interest rate
                - (360.0 / 90.0) * (
                        np.log(df_merged[fwd_col]) - np.log(df_merged[spot_col])
                )
                - (df_merged[usd_ir_col] / 100.0)  # foreign interest rate (USD)
        )

    ######################################
    # Rolling outlier cleanup (45-day window)
    ######################################
    window_size = 45
    for ccy in currencies:
        cip_col = f'CIP_{ccy}_ln'
        if cip_col not in df_merged.columns:
            continue

        # Rolling median over 45 days
        rolling_median = df_merged[cip_col].rolling(window_size).median()

        # Absolute deviation from median
        abs_dev = (df_merged[cip_col] - rolling_median).abs()

        # Rolling mean of abs_dev (proxy for MAD)
        rolling_mad = abs_dev.rolling(window_size).mean()

        # Mark outliers (abs_dev / mad >= 10) and replace with NaN
        outlier_mask = (abs_dev / rolling_mad) >= 10
        df_merged.loc[outlier_mask, cip_col] = np.nan

    # Create a separate DataFrame for the CIP columns
    cip_cols = [f'CIP_{c}_ln' for c in currencies if f'CIP_{c}_ln' in df_merged.columns]
    spreads = df_merged[cip_cols].copy()

    # Shorten column names for plotting
    spreads.columns = [c[4:7] for c in spreads.columns]  # e.g., CIP_AUD_ln -> AUD

    return df_merged.iloc[:, -8:]

def compute_cip_statistics(cip_data):
    """Compute CIP statistics from CIP data."""
    if cip_data is None or cip_data.empty:
        raise ValueError("Error: cip_data is empty or None. Check if compute_cip() is working correctly.")

    stats_dict = {}
    cip_columns = [col for col in cip_data.columns if col.startswith('CIP_') and col.endswith('_ln')]
    cip_df = cip_data[cip_columns]

    if cip_df.empty:
        raise ValueError("Error: cip_df is empty after filtering CIP columns.")

    stats_dict["overall_statistics"] = cip_df.describe()
    stats_dict["correlation_matrix"] = cip_df.corr()
    cip_data.index = pd.to_datetime(cip_data.index)
    stats_dict["annual_statistics"] = cip_df.resample('YE').agg(['mean', 'std', 'min', 'max'])

    return stats_dict


## Treasury Spot
```python
def task_clean_raw():
    """Run `clean_raw.ipynb` to generate necessary datasets in `_data`."""
    return {
        "actions": [
            jupyter_execute_notebook("clean_raw")
        ],
        "targets": [
            DATA_DIR / "treasury_df.csv",
            DATA_DIR / "ois_df.csv",
            DATA_DIR / "last_day_df.csv"
        ],
        "clean": True,
    }

In [ ]:
df_dates = pd.read_excel(data_file, sheet_name="T_SF", usecols="A", skiprows=6, header=None)
df_dates.columns = ["Date"]
df_dates["Date"] = pd.to_datetime(df_dates["Date"])

# Compute month, year, and day
df_dates["Mat_Month"] = df_dates["Date"].dt.month
df_dates["Mat_Year"] = df_dates["Date"].dt.year
df_dates = df_dates.sort_values("Date").reset_index(drop=True)

# For each month, keep the last date (mimicking: keep if mofd(Date) != mofd(Date[_n+1]))
df_last = df_dates.groupby([df_dates["Mat_Year"], df_dates["Mat_Month"]], as_index=False).agg({"Date": "last"})

# Compute the last day of the month
df_last["Mat_Day"] = df_last["Date"].dt.day

# Drop duplicates and keep required columns
df_matday = df_last[["Date", "Mat_Month", "Mat_Year", "Mat_Day"]].copy()
output_file = os.path.join(DATA_DIR, "last_day_df.csv") 
# Save as CSV
df_matday.to_csv(output_file, index=False)

# Display the final DataFrame
df_matday.head()


In [ ]:


# Load data from the Excel file
df = pd.read_excel(data_file, sheet_name="T_SF", skiprows=6, header=None)

# Define base column names
base_columns = ["Date"]
tenors = [10, 5, 2, 20, 30]  # Available tenors
versions = [1, 2]  # Nearby (1) and Deferred (2) contract versions

# Generate column names dynamically
col_names = ["Date"] + [
    f"{metric}_{v}_{tenor}" 
    for v in versions 
    for tenor in tenors 
    for metric in ["Implied_Repo", "Vol", "Contract", "Price"]
]

# Assign column names
df.columns = col_names

# Drop rows with missing dates
df = df.dropna(subset=["Date"])
df["Date"] = pd.to_datetime(df["Date"])

# Convert numeric columns
numeric_cols = [col for col in df.columns if col.startswith(("Implied_Repo", "Vol_", "Price_"))]
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

# **Sort columns alphabetically while keeping "Date" first**
sorted_columns = ["Date"] + sorted([col for col in df.columns if col != "Date"])
df = df[sorted_columns]

# Display sorted DataFrame
df.head()

treasury_df = df.copy()
output_file = os.path.join(DATA_DIR, "treasury_df.csv") 
# Save as CSV
# Optionally, save it to a file for later use
treasury_df.to_csv(output_file, index=False)  # Save as CSV


In [ ]:

# Load the Excel file
df_ois = pd.read_excel(ois_file, header=None)

# Drop the first 4 columns
df_ois = df_ois.iloc[:, 4:]

# Drop the top 3 rows and reset index
df_ois = df_ois.iloc[3:].reset_index(drop=True)

# Drop rows at index 1 and 2, then reset index again
df_ois = df_ois.drop(index=[1, 2]).reset_index(drop=True)

# Set the first row as the column headers, then drop it from the DataFrame
df_ois.columns = df_ois.iloc[0]
df_ois = df_ois[1:].reset_index(drop=True)

# Rename the first column to "Date"
df_ois.rename(columns={df_ois.columns[0]: "Date"}, inplace=True)

# Define the renaming mapping for OIS columns
rename_map = {
    "USSO1Z CMPN Curncy": "OIS_1W",
    "USSOA CMPN Curncy":  "OIS_1M",
    "USSOB CMPN Curncy":  "OIS_2M",
    "USSOC CMPN Curncy":  "OIS_3M",
    "USSOF CMPN Curncy":  "OIS_6M",
    "USSO1 CMPN Curncy":  "OIS_1Y",
    "USSO2 CMPN Curncy":  "OIS_2Y",
    "USSO3 CMPN Curncy":  "OIS_3Y",
    "USSO4 CMPN Curncy":  "OIS_4Y",
    "USSO5 CMPN Curncy":  "OIS_5Y",
    "USSO7 CMPN Curncy":  "OIS_7Y",
    "USSO10 CMPN Curncy": "OIS_10Y",
    "USSO15 CMPN Curncy": "OIS_15Y",
    "USSO20 CMPN Curncy": "OIS_20Y",
    "USSO30 CMPN Curncy": "OIS_30Y"
}

# Rename the OIS columns
df_ois.rename(columns=rename_map, inplace=True)

# Convert 'Date' column to datetime format
df_ois["Date"] = pd.to_datetime(df_ois["Date"], errors="coerce")

output_file = os.path.join(DATA_DIR, "ois_df.csv") 

# Save the cleaned data to CSV
df_ois.to_csv(output_file, index=False)

# Display the first few rows
df_ois.head()


```python

def task_generate_reference():
    """Run generate_reference.py to create reference.csv"""
    return {
        "actions": ["python ./src/generate_reference.py"],
        "targets": [DATA_DIR / "reference.csv"],
        "clean": True,
    }

In [ ]:
import os
import pandas as pd
from settings import config
import load_bases_data  # Ensure this module is correctly located in src/

# Get DATA_DIR from config
DATA_DIR = config("DATA_DIR")

# Ensure DATA_DIR exists
os.makedirs(DATA_DIR, exist_ok=True)

# Load the dataset
df = load_bases_data.load_combined_spreads_wide(data_dir=DATA_DIR)

# Forward-fill missing values (limit 5), then drop remaining NaNs
df = df.ffill(limit=5).dropna()

# Reindex columns in sorted order and filter only columns matching "Treasury_SF_*"
filtered_df = df.reindex(sorted(df.columns), axis=1).filter(regex="^Treasury_SF_")

# Define output file path
output_file = os.path.join(DATA_DIR, "reference.csv")

# Save the filtered DataFrame to CSV in DATA_DIR
filtered_df.to_csv(output_file, index=True)

# Display dataset information
print(filtered_df.info())

print(f"Reference dataset saved to {output_file}")


```python

def task_calc_treasury_data():
    """Run `calc_treasury_data.py` which processes Treasury SF data."""
    return {
        "actions": ["python ./src/calc_treasury_data.py"],
        "file_dep": [
            DATA_DIR / "treasury_df.csv",
            DATA_DIR / "ois_df.csv",
            DATA_DIR / "last_day_df.csv"
        ],
        "targets": [DATA_DIR / "treasury_sf_output.csv"],
        "clean": True,
    }

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import sys
import os

# Ensure 'src' is in sys.path
sys.path.append(os.path.abspath("./src"))  # Add 'src' to the path

# Now import config from settings.py
from settings import config  # <- FIXED

# Set data directory
DATA_DIR = config("DATA_DIR")
MANUAL_DATA_DIR = config("MANUAL_DATA_DIR")
OUTPUT_DIR = config("OUTPUT_DIR")

def parse_contract_date(contract_str):
    """
    Parse contract string to extract month and year.
    E.g., "DEC 21" -> (12, 2021)
    """
    if pd.isna(contract_str) or not isinstance(contract_str, str):
        return None, None
    month_abbr = contract_str[:3].upper()
    year_str = contract_str[4:6]
    month_map = {'DEC': 12, 'MAR': 3, 'JUN': 6, 'SEP': 9}
    month = month_map.get(month_abbr, np.nan)
    try:
        year = int(year_str) + 2000
    except:
        year = np.nan
    return month, year

def interpolate_ois(ttm, ois_1w, ois_1m, ois_3m, ois_6m, ois_1y):
    """Interpolate the OIS rate based on TTM (in days)."""
    if ttm <= 7:
        return ois_1w
    elif 7 < ttm <= 30:
        return ((30 - ttm) / 23) * ois_1w + ((ttm - 7) / 23) * ois_1m
    elif 30 < ttm <= 90:
        return ((90 - ttm) / 60) * ois_1m + ((ttm - 30) / 60) * ois_3m
    elif 90 < ttm <= 180:
        return ((180 - ttm) / 90) * ois_3m + ((ttm - 90) / 90) * ois_6m
    else:
        return ((360 - ttm) / 180) * ois_6m + ((ttm - 180) / 180) * ois_1y

def rolling_outlier_flag(df, group_col, date_col, value_col, window_days=45, threshold=10):
    """
    Flag outliers using a rolling window (±45 days) per group.
    """
    df = df.copy()
    df['bad_price'] = False
    df[date_col] = pd.to_datetime(df[date_col])
    df.sort_values(date_col, inplace=True)
    
    for name, group in df.groupby(group_col):
        for idx, row in group.iterrows():
            curr_date = row[date_col]
            window_mask = (group[date_col] >= curr_date - timedelta(days=window_days)) & \
                          (group[date_col] <= curr_date + timedelta(days=window_days)) & \
                          (group.index != idx)
            window_vals = group.loc[window_mask, value_col]
            if len(window_vals) > 0:
                median_val = window_vals.median()
                abs_dev = abs(row[value_col] - median_val)
                mad = window_vals.subtract(median_val).abs().mean()
                if mad > 0 and (abs_dev / mad) >= threshold:
                    df.at[idx, 'bad_price'] = True
    return df

def calc_treasury():
    # Load intermediate data
    data_dir = DATA_DIR

    # Load the required CSV files from _data
    treasury_file = os.path.join(data_dir, "treasury_df.csv")
    ois_file = os.path.join(data_dir, "ois_df.csv")
    last_day_file = os.path.join(data_dir, "last_day_df.csv")

    # Read the datasets
    df = pd.read_csv(treasury_file)
    df_ois = pd.read_csv(ois_file)
    last_day_df = pd.read_csv(last_day_file)

    # Convert date columns to datetime format
    df["Date"] = pd.to_datetime(df["Date"])
    df_ois["Date"] = pd.to_datetime(df_ois["Date"])
    last_day_df["Date"] = pd.to_datetime(last_day_df["Date"])

    # -------------------------
    # Reshape from wide to long format
    stubnames = ["Contract_1", "Contract_2", "Implied_Repo_1", "Implied_Repo_2",
                 "Vol_1", "Vol_2", "Price_1", "Price_2"]
    df_long = pd.wide_to_long(df, stubnames=stubnames, i="Date", j="Tenor", sep="_", suffix=r'\d+').reset_index()

    # Filter dates > June 22, 2004
    cutoff_date = datetime(2004, 6, 22)
    df_long = df_long[df_long["Date"] > cutoff_date].copy()
    
    # -------------------------
    # Compute time-to-maturity for contracts v=1 and v=2
    for v in [1, 2]:
        contract_col = f"Contract_{v}"
        ttm_col = f"TTM_{v}"
        mat_date_col = f"Mat_Date_{v}"
        
        # Parse contract string to get month and year
        df_long[[f"Mat_Month_{v}", f"Mat_Year_{v}"]] = df_long[contract_col].apply(
            lambda s: pd.Series(parse_contract_date(s))
        )
        
        # Merge with last_day_df to get the day-of-month
        df_long = df_long.merge(last_day_df, left_on=[f"Mat_Month_{v}", f"Mat_Year_{v}"], 
                                right_on=["Mat_Month", "Mat_Year"], how="left", suffixes=("", f"_{v}"))
        # For specific contracts without a business day, set Mat_Day = 31
        cond_special = df_long[contract_col].isin(["DEC 21", "MAR 22"])
        df_long.loc[cond_special, "Mat_Day"] = 31
        
        def make_mat_date(row):
            try:
                return datetime(int(row[f"Mat_Year_{v}"]), int(row[f"Mat_Month_{v}"]), int(row["Mat_Day"]))
            except Exception:
                return pd.NaT
        df_long[mat_date_col] = df_long.apply(make_mat_date, axis=1)
        df_long[ttm_col] = (df_long[mat_date_col] - df_long["Date"]).dt.days
        
        # Clean up temporary columns
        df_long.drop(columns=[f"Mat_Month_{v}", f"Mat_Year_{v}", "Mat_Month", "Mat_Year", "Mat_Day"], 
                      inplace=True, errors='ignore')
    
    # -------------------------
    
    # Merge with USD OIS Rates on Date
    df_ois['Date'] = pd.to_datetime(df_ois['Date'])
    df_long = df_long.merge(df_ois, left_on="Date", right_on="Date", how="left")
    # df_long.drop(columns=["Date"], inplace=True)
    


    # -------------------------
    # Interpolate OIS rates for contracts v=1 and v=2
    for v in [1, 2]:
        ttm_col = f"TTM_{v}"
        ois_col = f"OIS_{v}"
        df_long[ois_col] = df_long.apply(lambda row: 
                                         interpolate_ois(row[ttm_col],
                                                         row.get("OIS_1W", np.nan),
                                                         row.get("OIS_1M", np.nan),
                                                         row.get("OIS_3M", np.nan),
                                                         row.get("OIS_6M", np.nan),
                                                         row.get("OIS_1Y", np.nan)
                                                        ) if pd.notnull(row[ttm_col]) else np.nan, axis=1)
    
    # -------------------------
    # Compute Treasury arbitrage spreads
    df_long["Arb_N"] = (df_long["Implied_Repo_1"] - df_long["OIS_1"]) * 100
    df_long["Arb_D"] = (df_long["Implied_Repo_2"] - df_long["OIS_2"]) * 100
    df_long["arb"] = df_long["Arb_D"]   # Use deferred contract
    
    # Outlier cleanup: flag observations based on a 45-day rolling window.
    df_long = rolling_outlier_flag(df_long, group_col="Tenor", date_col="Date", value_col="arb",
                                   window_days=45, threshold=10)
    df_long.loc[df_long["bad_price"] & df_long["arb"].notnull(), "arb"] = np.nan
    
    # Drop rows without trading volume in deferred contract (Vol_2)
    df_long = df_long[df_long["Vol_2"].notnull()].copy()
    
    # -------------------------
    # Plot arbitrage spread for selected tenors.
    output_dir = OUTPUT_DIR 
    os.makedirs(output_dir, exist_ok=True)
    for tenor in [2, 5, 10, 20, 30]:
        df_plot = df_long[df_long["Tenor"] == str(tenor)]
        if df_plot.empty:
            continue
        plt.figure(figsize=(10, 5))
        plt.plot(df_plot["Date"], df_plot["arb"], label=f"Tenor = {tenor} years")
        plt.ylabel("Arbitrage Spread (bps)")
        plt.xlabel("")
        plt.title(f"Tenor = {tenor} years")
        plt.legend()
        plt.tight_layout()
        plot_path = os.path.join(output_dir, f"arbitrage_spread_{tenor}.pdf")
        plt.savefig(plot_path)
        plt.close()
        print(f"Saved plot: {plot_path}")
    
    # -------------------------
    # Prepare final output
    df_long["T_SF_Rf"] = df_long["Implied_Repo_2"] * 100
    df_long.loc[df_long["bad_price"] & df_long["T_SF_Rf"].notnull(), "T_SF_Rf"] = np.nan
    df_long["rf_ois_t_sf_mat"] = df_long["OIS_2"] * 100
    df_long["T_SF_TTM"] = df_long["TTM_2"]
    df_out = df_long[["Date", "Tenor", "T_SF_Rf", "rf_ois_t_sf_mat", "T_SF_TTM"]].copy()
    
    # Reshape output to wide format (one row per date)
    df_wide = df_out.pivot(index="Date", columns="Tenor")
    df_wide.columns = ['_'.join([str(c) for c in col]).strip() for col in df_wide.columns.values]
    df_wide.reset_index(inplace=True)
    
    # Rename columns to match output convention
    rename_dict = {
        "T_SF_Rf_2": "tfut_2_rf",
        "T_SF_Rf_5": "tfut_5_rf",
        "T_SF_Rf_10": "tfut_10_rf",
        "T_SF_Rf_20": "tfut_20_rf",
        "T_SF_Rf_30": "tfut_30_rf",
        "T_SF_TTM_2": "tfut_2_ttm",
        "T_SF_TTM_5": "tfut_5_ttm",
        "T_SF_TTM_10": "tfut_10_ttm",
        "T_SF_TTM_20": "tfut_20_ttm",
        "T_SF_TTM_30": "tfut_30_ttm",
        "rf_ois_t_sf_mat_2": "tfut_2_ois",
        "rf_ois_t_sf_mat_5": "tfut_5_ois",
        "rf_ois_t_sf_mat_10": "tfut_10_ois",
        "rf_ois_t_sf_mat_20": "tfut_20_ois",
        "rf_ois_t_sf_mat_30": "tfut_30_ois"
    }
    df_wide.rename(columns=rename_dict, inplace=True)

    df_wide["Treasury_SF_2Y"] = df_wide["tfut_2_rf"] - df_wide["tfut_2_ois"]
    df_wide["Treasury_SF_5Y"] = df_wide["tfut_5_rf"] - df_wide["tfut_5_ois"]
    df_wide["Treasury_SF_10Y"] = df_wide["tfut_10_rf"] - df_wide["tfut_10_ois"]
    df_wide["Treasury_SF_20Y"] = df_wide["tfut_20_rf"] - df_wide["tfut_20_ois"]
    df_wide["Treasury_SF_30Y"] = df_wide["tfut_30_rf"] - df_wide["tfut_30_ois"]
    # Select relevant columns
    df_out = df_wide[["Date", "Treasury_SF_2Y", "Treasury_SF_5Y", "Treasury_SF_10Y", "Treasury_SF_20Y", "Treasury_SF_30Y"]].copy()
    df_out.fillna(method='ffill', inplace=True)

    
    # Save final output as Stata .dta file (or CSV if preferred)
    output_file = os.path.join(DATA_DIR, "treasury_sf_output.csv")
    df_out.to_csv(output_file, index=False)




    print(f"Final output saved to {output_file}")
    
if __name__ == '__main__':
    calc_treasury()


# Generate Figures

## TIPS Treasury Arbitrage:
```python
def task_generate_figures():
    """ """
    file_dep = [
        "./src/generate_figures.py",
        "./src/generate_latex_table.py",
    ]
    file_output = [
        "tips_treasury_spreads.png",
        "tips_treasury_summary_stats.csv",
        'tips_treasury_summary_table.tex'
    ]
    targets = [OUTPUT_DIR / file for file in file_output]

    return {
        "actions": [
            "ipython ./src/generate_figures.py",
            "ipython ./src/generate_latex_table.py",
        ],
        "targets": targets,
        "file_dep": file_dep,
        "clean": [],
    }
```


## CIP 
``` python

def task_rename_plots():
    """Run CIP analysis and rename output plots correctly."""

    import re

    def rename_output_files():
        """Rename automatically generated plots to expected filenames based on a pattern match."""
        output_dir = OUTPUT_DIR / "main_cip_files"
        Path(output_dir).mkdir(parents=True, exist_ok=True)

        file_patterns = {
            r"main_cip_\d+_0.png": "cip_spread_plot_replication.png",
            r"main_cip_\d+_1.png": "cip_spread_plot_2025.png",
        }

        matched_files = set()  # Track matched files

        for old_file in output_dir.glob("main_cip_*.png"):
            for pattern, new_name in file_patterns.items():
                if re.match(pattern, old_file.name):
                    new_path = output_dir / new_name

                    # Check if the file already exists and avoid overwriting
                    if new_path.exists():
                        new_path = output_dir / "cip_spread_2025.png"

                    old_file.rename(new_path)
                    print(f"Renamed {old_file} → {new_path}")
                    matched_files.add(old_file.name)
                    # No break here so all files get renamed

        # Check for missing files
        for old_file in output_dir.glob("main_cip_*.png"):
            if old_file.name not in matched_files:
                print(f"Warning: No match found for {old_file}")

    return {
        "actions": [
            "cd src && ipython cip_analysis.py",
            rename_output_files  # Rename after execution
        ],
        "file_dep": [
            "./src/cip_analysis.py",
            "./src/pull_bloomberg_cip_data.py",
            str(MANUAL_DATA_DIR / "CIP_2025.xlsx"),
        ],
        "targets": [
            str(OUTPUT_DIR / "main_cip_files" / "cip_spread_plot_replication.png"),
            str(OUTPUT_DIR / "main_cip_files" / "cip_spread_plot_2025.png"),
        ],
        "task_dep": ["download_cip_data"],
        "clean": True,
    }


# Convert Notebooks 


## TIPS Treasury Arbitrage:

```python
notebook_tasks = {
    "arb_replication.ipynb": {
        "file_dep": [
            "./src/generate_figures.py"
        ],
        "targets": [],
    }
}

def task_convert_notebooks_to_scripts():
    """Convert notebooks to script form to detect changes to source code rather
    than to the notebook's metadata.
    """
    build_dir = Path(OUTPUT_DIR)

    for notebook in notebook_tasks.keys():
        notebook_name = notebook.split(".")[0]
        yield {
            "name": notebook,
            "actions": [
                jupyter_clear_output(notebook_name),
                jupyter_to_python(notebook_name, build_dir),
            ],
            "file_dep": [Path("./src") / notebook],
            "targets": [OUTPUT_DIR / f"_{notebook_name}.py"],
            "clean": True,
            "verbosity": 0,
        }
```

## Equity Spot Futures:
```python
notebook_tasks = {
    "01_OIS_Data_Processing.ipynb": {
        "file_dep": ["./src/settings.py","./src/pull_bloomberg_data.py", "./src/OIS_data_processing.py"],
        "targets": [OUTPUT_DIR / 'ois_3m_rolling_statistics.png',
                    OUTPUT_DIR / 'ois_3m_rate_time_series.png',
                    OUTPUT_DIR / "ois_summary_statistics.tex"],
    },
    "02_Futures_Data_Processing.ipynb": {
        "file_dep": ["./src/settings.py","./src/pull_bloomberg_data.py", "./src/futures_data_processing.py"],
        "targets": [OUTPUT_DIR / "es1_contract_roll_pattern.png",
                    OUTPUT_DIR / "es1_ttm_distribution.png",
                    OUTPUT_DIR / "futures_prices_by_index.png",],
    },
    "03_Spread_Calculations.ipynb": {
        "file_dep": [
            "./src/settings.py",
            "./src/pull_bloomberg_data.py", 
            "./src/futures_data_processing.py",
            "./src/OIS_data_processing.py",
            "./src/Spread_calculations.py"
        ],
        "targets": [],
    },
}


def task_convert_notebooks_to_scripts():
    """Convert notebooks to script form to detect changes to source code rather
    than to the notebook's metadata.
    """
    build_dir = Path(OUTPUT_DIR)

    for notebook in notebook_tasks.keys():
        notebook_name = notebook.split(".")[0]
        yield {
            "name": notebook,
            "actions": [
                jupyter_clear_output(notebook_name),
                jupyter_to_python(notebook_name, build_dir),
            ],
            "file_dep": [Path("./src") / notebook],
            "targets": [OUTPUT_DIR / f"_{notebook_name}.py"],
            "clean": True,
            "verbosity": 0,
        }
```

## Market Expectations:

```python


notebook_tasks = {
    "01_Market_Expectations_In_The_Cross-Section_Of_Present_Values_Final.ipynb": {
        "file_dep": [],
        "targets": [],
    },
    "run_regressions.ipynb": {
        "file_dep": [],
        "targets": [],
    },
}

def task_convert_notebooks_to_scripts():
    """Convert notebooks to script form to detect changes to source code rather
    than to the notebook's metadata.
    """
    build_dir = Path(OUTPUT_DIR)

    for notebook in notebook_tasks.keys():
        notebook_name = notebook.split(".")[0]
        yield {
            "name": notebook,
            "actions": [
                jupyter_clear_output(notebook_name),
                jupyter_to_python(notebook_name, build_dir),
            ],
            "file_dep": [Path("./src") / notebook],
            "targets": [OUTPUT_DIR / f"_{notebook_name}.py"],
            "clean": True,
            "verbosity": 0,
        }

# Run Notebooks


## TIPS Treasury Arbitrage:

```python
# fmt: off
def task_run_notebooks():
    """Preps the notebooks for presentation format.
    Execute notebooks if the script version of it has been changed.
    """
    for notebook in notebook_tasks.keys():
        notebook_name = notebook.split(".")[0]
        yield {
            "name": notebook,
            "actions": [
                """python -c "import sys; from datetime import datetime; print(f'Start """ + notebook + """: {datetime.now()}', file=sys.stderr)" """,
                jupyter_execute_notebook(notebook_name),
                jupyter_to_html(notebook_name),
                copy_file(
                    Path("./src") / f"{notebook_name}.ipynb",
                    OUTPUT_DIR / f"{notebook_name}.ipynb",
                    mkdir=True,
                ),
                jupyter_clear_output(notebook_name),
                # jupyter_to_python(notebook_name, build_dir),
                """python -c "import sys; from datetime import datetime; print(f'End """ + notebook + """: {datetime.now()}', file=sys.stderr)" """,
            ],
            "file_dep": [
                OUTPUT_DIR / f"_{notebook_name}.py",
                *notebook_tasks[notebook]["file_dep"],
            ],
            "targets": [
                OUTPUT_DIR / f"{notebook_name}.html",
                OUTPUT_DIR / f"{notebook_name}.ipynb",
                *notebook_tasks[notebook]["targets"],
            ],
            "clean": True,
        }
```

## Equity Spot Futures:
```python
# fmt: off
def task_run_notebooks():
    """Preps the notebooks for presentation format.
    Execute notebooks if the script version of it has been changed.
    """
    for notebook in notebook_tasks.keys():
        notebook_name = notebook.split(".")[0]
        yield {
            "name": notebook,
            "actions": [
                """python -c "import sys; from datetime import datetime; print(f'Start """ + notebook + """: {datetime.now()}', file=sys.stderr)" """,
                jupyter_execute_notebook(notebook_name),
                jupyter_to_html(notebook_name),
                copy_file(
                    Path("./src") / f"{notebook_name}.ipynb",
                    OUTPUT_DIR / f"{notebook_name}.ipynb",
                    mkdir=True,
                ),
                copy_file(
                    Path("./src") / f"{notebook_name}.ipynb",
                    Path("./_docs/notebooks/") / f"{notebook_name}.ipynb",
                    mkdir=True,
                ),
                jupyter_clear_output(notebook_name),
                # jupyter_to_python(notebook_name, build_dir),
                """python -c "import sys; from datetime import datetime; print(f'End """ + notebook + """: {datetime.now()}', file=sys.stderr)" """,
            ],
            "file_dep": [
                OUTPUT_DIR / f"_{notebook_name}.py",
                *notebook_tasks[notebook]["file_dep"],
            ],
            "targets": [
                OUTPUT_DIR / f"{notebook_name}.html",
                OUTPUT_DIR / f"{notebook_name}.ipynb",
                *notebook_tasks[notebook]["targets"],
            ],
            "clean": True,
        }
```

## CIP 
```python
        
def task_run_notebooks():
    """Execute Jupyter notebooks and convert them to HTML & LaTeX."""
    notebook = "main_cip"

    return {
        "actions": [
            jupyter_execute_notebook(notebook),
            jupyter_to_html(notebook),
            jupyter_to_latex(notebook),
            # convert_html_to_png  # Move PNGs and process files
        ],
        "file_dep": ["./src/main_cip.ipynb", str(MANUAL_DATA_DIR / "CIP_2025.xlsx")],
        "targets": [
            str(OUTPUT_DIR /"html_files"/ "main_cip.html"),
            str(OUTPUT_DIR / "main_cip.tex"),
        ],
        "task_dep": ["download_cip_data"],
        "clean": True,
    }
```

## Market Expectations
```python

# fmt: off
def task_run_notebooks():
    """Preps the notebooks for presentation format.
    Execute notebooks if the script version of it has been changed.
    """
    for notebook in notebook_tasks.keys():
        notebook_name = notebook.split(".")[0]
        yield {
            "name": notebook,
            "actions": [
                """python -c "import sys; from datetime import datetime; print(f'Start """ + notebook + """: {datetime.now()}', file=sys.stderr)" """,
                jupyter_execute_notebook(notebook_name),
                jupyter_to_html(notebook_name),
                copy_file(
                    Path("./src") / f"{notebook_name}.ipynb",
                    OUTPUT_DIR / f"{notebook_name}.ipynb",
                    mkdir=True,
                ),
                jupyter_clear_output(notebook_name),
                # jupyter_to_python(notebook_name, build_dir),
                """python -c "import sys; from datetime import datetime; print(f'End """ + notebook + """: {datetime.now()}', file=sys.stderr)" """,
            ],
            "file_dep": [
                OUTPUT_DIR / f"_{notebook_name}.py",
                *notebook_tasks[notebook]["file_dep"],
            ],
            "targets": [
                OUTPUT_DIR / f"{notebook_name}.html",
                OUTPUT_DIR / f"{notebook_name}.ipynb",
                *notebook_tasks[notebook]["targets"],
            ],
            "clean": True,
        }

## Treasury Spot
```python


def task_process_treasury_data_notebook():
    """Optionally run `process_treasury_data.ipynb` if needed."""
    return {
        "actions": [
            jupyter_execute_notebook("process_treasury_data")
        ],
        "file_dep": [DATA_DIR / "treasury_sf_output.csv"],
        "clean": True,
    }

def task_plot_exploration():
    """Run `01_explore_basis_trade_data_new.ipynb` for visualization."""
    return {
        "actions": [
            jupyter_execute_notebook("01_explore_basis_trade_data_new")
        ],
        "file_dep": [DATA_DIR / "treasury_sf_output.csv"],
        "clean": True,
    }

# LaTeX compiliation


## TIPS Treasury Arbitrage:

```python
def task_compile_latex_docs():
    """Compile the LaTeX documents to PDFs"""
    file_dep = [
        "./reports/report.tex",
        "./reports/my_article_header.sty",      # style 
        #"./reports/slides_example.tex",
        #`"./reports/my_beamer_header.sty",       # style
        "./reports/my_common_header.sty",       # style
        # "./reports/report_simple_example.tex",
        # "./reports/slides_simple_example.tex",
        "./src/generate_figures.py",
        "./src/generate_latex_table.py",
    ]
    targets = [
        "./reports/report.pdf",
        #"./reports/slides_example.pdf",
        # "./reports/report_simple_example.pdf",
        # "./reports/slides_simple_example.pdf",
    ]

    return {
        "actions": [
            # My custom LaTeX templates
            "latexmk -xelatex -halt-on-error -cd ./reports/report.tex",  # Compile
            "latexmk -xelatex -halt-on-error -c -cd ./reports/report.tex",  # Clean
      ],
        "targets": targets,
        "file_dep": file_dep,
        "clean": True,
    }
```


## Equity Spot Futures
```python
def task_compile_latex_docs():
    """Compile the LaTeX documents to PDFs"""
    file_dep = [
        "./reports/report.tex"
    ]
    targets = [
        "./reports/report.pdf"
    ]

    return {
        "actions": [
            "latexmk -xelatex -halt-on-error -cd ./reports/report.tex",  # Compile
            "latexmk -xelatex -halt-on-error -c -cd ./reports/report.tex"  # Clean
        ],
        "targets": targets,
        "file_dep": file_dep,
        "clean": True,
    }
```
## Market Expectations
```python
def task_compile_latex_docs():
    """Compile the LaTeX documents to PDFs"""
    file_dep = [
        "./reports/project.tex",
    ]
    targets = [
        "./reports/project.pdf",
    ]

    return {
        "actions": [
            "latexmk -xelatex -halt-on-error -cd ./reports/project.tex",  # Compile
            "latexmk -xelatex -halt-on-error -c -cd ./reports/project.tex",  # Clean
        ],
        "targets": targets,
        "file_dep": file_dep,
        "clean": True,
    }
```
## Treasury spot
```python
def task_latex_to_document():
    return {
        'actions': ['python src/latex_to_document.py reports/Final_Report.tex'],
        'file_dep': ['src/latex_to_document.py', 'reports/Final_Report.tex'],
        'targets': ['reports/Final_Report.pdf'],
        'clean': True,
    }

# Sphinx


## Equity Spot Futures

```python

notebook_sphinx_pages = [
    "./_docs/_build/html/notebooks/" + notebook.split(".")[0] + ".html"
    for notebook in notebook_tasks.keys()
]
sphinx_targets = [
    "./_docs/_build/html/index.html",
    *notebook_sphinx_pages
]


def copy_docs_src_to_docs():
    """
    Copy all files and subdirectories from the docs_src directory to the _docs directory.
    This function loops through all files in docs_src and copies them individually to _docs,
    preserving the directory structure. It does not delete the contents of _docs beforehand.
    """
    src = Path("docs_src")
    dst = Path("_docs")

    # Ensure the destination directory exists
    dst.mkdir(parents=True, exist_ok=True)

    # Loop through all files and directories in docs_src
    for item in src.rglob("*"):
        relative_path = item.relative_to(src)
        target = dst / relative_path
        if item.is_dir():
            target.mkdir(parents=True, exist_ok=True)
        else:
            shutil.copy2(item, target)


def copy_docs_build_to_docs():
    """
    Copy all files and subdirectories from _docs/_build/html to docs.
    This function copies each file individually while preserving the directory structure.
    It does not delete any existing contents in docs.
    After copying, it creates an empty .nojekyll file in the docs directory.
    """
    src = Path("_docs/_build/html")
    dst = Path("docs")
    dst.mkdir(parents=True, exist_ok=True)

    # Loop through all files and directories in src
    for item in src.rglob("*"):
        relative_path = item.relative_to(src)
        target = dst / relative_path
        if item.is_dir():
            target.mkdir(parents=True, exist_ok=True)
        else:
            target.parent.mkdir(parents=True, exist_ok=True)
            shutil.copy2(item, target)

    # Touch an empty .nojekyll file in the docs directory.
    (dst / ".nojekyll").touch()


def task_compile_sphinx_docs():
    """Compile Sphinx Docs"""
    notebook_scripts = [
        OUTPUT_DIR / ("_" + notebook.split(".")[0] + ".py")
        for notebook in notebook_tasks.keys()
    ]
    file_dep = [
        "./docs_src/conf.py",
        "./docs_src/index.md",
        *notebook_scripts,
    ]

    return {
        "actions": [
            copy_docs_src_to_docs,
            "sphinx-build -M html ./_docs/ ./_docs/_build",
            copy_docs_build_to_docs,
        ],
        "targets": sphinx_targets,
        "file_dep": file_dep,
        "task_dep": ["run_notebooks"],
        "clean": True,
    }


```

## CIP 

```python

def task_generate_paper():
    """Generate a LaTeX paper from the copied Jupyter Notebook."""
    paper_notebook = PUBLISH_DIR / "paper.ipynb"
    paper_tex = PUBLISH_DIR / "paper.tex"


    return {
        "actions": [
            copy_notebook,  # Copy first
            f"jupyter nbconvert --execute --to notebook --inplace --ExecutePreprocessor.allow_errors=True \"{paper_notebook}\"",
            f"jupyter nbconvert --to latex --output-dir=\"{PUBLISH_DIR}\" \"{paper_notebook}\"",
            f"pdflatex -output-directory=\"{PUBLISH_DIR}\" \"{paper_tex}\"",
            # f"bibtex \"{paper_tex.with_suffix('')}\"",  # Keep commented if no bibliography
            f"pdflatex -output-directory=\"{PUBLISH_DIR}\" \"{paper_tex}\"",
            f"pdflatex -output-directory=\"{PUBLISH_DIR}\" \"{paper_tex}\""
        ],
        "file_dep": [],
        "targets": [str(paper_tex)],
        "task_dep": [],
        "clean": True,
    }

def task_clean_reports():
    """Remove unnecessary output files."""
    files_to_remove = [
        PUBLISH_DIR / "paper.aux",
        PUBLISH_DIR / "paper.log",
        PUBLISH_DIR / "paper.out",
        PUBLISH_DIR / "spread_plot_rep.pdf"
    ]

    def remove_files():
        for file in files_to_remove:
            if file.is_file():
                file.unlink()
                print(f"Removed {file}")
            else:
                print(f"File not found: {file}")

    return {
        "actions": [remove_files],
        "clean": True,
    }
